In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm , skew
from scipy.special import boxcox
import os
from scipy import stats
from sklearn.model_selection import KFold ,cross_val_score
import warnings  ### 煩 XD
warnings.filterwarnings("ignore")
from category_encoders.target_encoder import TargetEncoder
from category_encoders.leave_one_out import LeaveOneOutEncoder
import xgboost as xgb
import lightgbm as lgb
import catboost as cat
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [2]:
city_data_list_all = os.listdir('../DATAS/DATAS_Hsin/')
city_data_list_all

['HsinChuHouseDataBySASClearMRTLine',
 'KaoHsiungHouseDataBySASClearMRTLine',
 'NewTaipeiHouseDataBySASClearMRTLine',
 'TaiChungHouseDataBySASClearMRTLine',
 'TainanHouseDataBySASClearMRTLine',
 'TaipeiHouseDataBySASClearMRTLine',
 'TaoYuanHouseDataBySASClearMRTLine']

In [177]:
def load_city_data (city_num) :
    city_i = pd.read_csv('../DATAS/DATAS_Hsin/{}/{}.csv'.format(city_data_list_all[city_num],city_data_list_all[city_num]))
    
    return city_i

In [178]:
len_hsin = len(load_city_data(0))
len_kao = len(load_city_data(1))
len_newtaipei = len(load_city_data(2))
len_taichung = len(load_city_data(3))
len_tainan = len(load_city_data(4))
len_taipei = len(load_city_data(5))
len_taoyuan = len(load_city_data(6))

In [179]:
data_all = pd.DataFrame()
for city in city_data_list_all :
    
    data = pd.read_csv('../DATAS/DATAS_Hsin/{}/{}.csv'.format(city,city))
    
    data_all = pd.concat([data_all,data],axis=0)

In [180]:
city_hsin = data_all.iloc[:len_hsin,:]
city_kao = data_all.iloc[len_hsin:len_hsin+len_kao,:]
city_newtaipei = data_all.iloc[len_hsin+len_kao:len_hsin+len_kao+len_newtaipei,:]
city_taichung = data_all.iloc[len_hsin+len_kao+len_newtaipei:len_hsin+len_kao+len_newtaipei+len_taichung,:]
city_tainan = data_all.iloc[len_hsin+len_kao+len_newtaipei+len_taichung:len_hsin+len_kao+len_newtaipei+len_taichung+len_tainan,:]
city_taipei = data_all.iloc[len_hsin+len_kao+len_newtaipei+len_taichung+len_tainan:len_hsin+len_kao+len_newtaipei+len_taichung+len_tainan+len_taipei,:]
city_taoyuan = data_all.iloc[len_hsin+len_kao+len_newtaipei+len_taichung+len_tainan+len_taipei:,:]


# 完美切割   by time 106年後

In [182]:
data_all_re.shape

(573415, 171)

In [183]:
train_set = data_all_re[data_all_re['TRADE_YM'] <= 10700]

In [184]:
train_set = data_all_re[data_all_re['TRADE_YM'] <= 10700]
test_set = data_all_re[data_all_re['TRADE_YM'] > 10700]

In [185]:
print(train_set.shape,test_set.shape)

(476527, 171) (96888, 171)


In [85]:
test_set_ans = test_set['BUILD_U_PRICE.ADJ']
test_set = test_set.drop('BUILD_U_PRICE.ADJ',axis=1)

In [86]:
data_all_kaggle = pd.concat([train_set,test_set])

In [299]:
ans = lig.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))

In [302]:
ans.shape

(95531,)

In [301]:
test_set_ans.shape

(191237,)

# preprocessing

In [186]:
from house_preprocess import *

In [187]:
def run_all_code (data_all) :

    
    def get_target_encoding_cate (df, test, target, nd_target_list):
        tar = TargetEncoder(smoothing=0.987).fit(df[nd_target_list], df[target])
        df[nd_target_list] = tar.transform(df[nd_target_list])
        test[nd_target_list] = tar.transform(test[nd_target_list])

        return df, test


    def get_target_encoding_int(df, test, target, nd_target_list):
        tar = TargetEncoder(smoothing=0.987).fit(df[nd_target_list].astype('str'), df[target])
        df[nd_target_list] = tar.transform(df[nd_target_list])
        test[nd_target_list] = tar.transform(test[nd_target_list])

        return df, test
    data_all_col = data_all.columns.tolist()
    data_all_na_col = []
    for col in data_all_col :
        if data_all[col].isna().sum() == 0 :
            pass
        else :
            data_all_na_col.append(col)
            
    ### fill na data
    data_all = fill_address_na(data_all,'ADDR')
    data_all = fill_address_na(data_all,'VILLAGE')
    data_all = fill_address_na(data_all,'OTROAD')
    data_all = new_address(data_all,'ADDR','ROAD')
    data_all = fill_all_disance(data_all,'BUILD_U_PRICE.ADJ',data_all_na_col)
    ### feature cross
    data_all = com_ornot(data_all,'COMMUNITY_NAME','IS_COMMUNITY')
    data_all = top_floor(data_all,'TRANS_FLOOR','TOTAL_FLOOR_NUM','TOP_FLOOR')
    data_all = get_month_build(data_all,'BUILD_DATE','BUILD_MONTH')
    data_all = get_month_trad(data_all,'TRADE_YM','TRAD_MONTH')
    data_all = get_years_build(data_all,'BUILD_DATE','BUILD_YEARS')
    data_all = get_years_trad(data_all,'TRADE_YM','TRAD_YEARS')
    data_all = cov_all_distance_log1p(data_all,data_all_col)
    data_all = cut_lat(data_all,'LATITUDE','lat_cut')
    data_all = cut_lat(data_all,'LONGITUDE','lon_cut')
    data_all = cross_col(data_all,'LATITUDE','LONGITUDE','cross_latlon')
    ### math tranlate
    data_all['LATITUDE.sqr'] = data_all['LATITUDE']**2
    data_all['LONGITUDE.sqr'] = data_all['LONGITUDE']**2
    data_all['HOUSE_YEAR.sqr'] = data_all['HOUSE_YEAR'] **2
    data_all['BUILD_AREA.sqr'] = data_all['BUILD_AREA'] **2
    data_all['TOTAL_FLOOR.sqr'] = data_all['TOTAL_FLOOR'] **2
    ### cip feature
    data_all['area_ratio'] = data_all['LAND_AREA']  / data_all['BUILD_AREA.ADJ']
    data_all['floor_ratio'] = data_all['TRANS_FLOOR']  / data_all['TOTAL_FLOOR_NUM']
    data_all['ave_tra_dis'] = (data_all['MRT_DISTANCE_SQL'] + data_all['HWAY_DISTANCE_SQL'] + data_all['RAIL_DISTANCE_SQL']) /3
    data_all['ave_ent_dis'] = (data_all['PARK_DISTANCE_SQL'] + data_all['CINEMA_DISTANCE']) /2
    data_all['ave_lif_dis'] = (data_all['MARKET_DISTANCE'] + data_all['STORE_DISTANCE_SQL'] + data_all['OILSTATION_DISTANCE']
                               + data_all['FASTFOOD_DISTANCE'] +data_all['CNVCSTORE_DISTANCE'] )/5 
    data_all['ave_health_dis'] = (data_all['HOSPITAL_DISTANCE_SQL'] + data_all['PHARMACY_DISTANCE']) /2
    data_all['ave_lif_tra_dis'] = (data_all['ave_lif_dis'] + data_all['ave_tra_dis'] ) /2
    data_all['ave_ent_tra_dis'] = (data_all['ave_ent_dis'] + data_all['ave_tra_dis'] ) / 2
    data_all['ave_lif_ent_dis']= (data_all['ave_lif_dis'] + data_all['ave_ent_dis'] ) /2
    data_all['ave_tra_sc'] = (data_all['MRT_SCORE'] + data_all['HWAY_SCORE'] + data_all['RAIL_SCORE']) /3
    data_all['ave_ent_sc'] = (data_all['PARK_SCORE'] + data_all['CINEMA_SCORE']) /2
    data_all['ave_lif_sc'] = (data_all['MARKET_SCORE'] + data_all['STORE_SCORE'] + data_all['FASTFOOD_SCORE'] + data_all['CNVCSTORE_SCORE']) /4 
    data_all['ave_health_sc'] = (data_all['HOSPITAL_SCORE'] + data_all['PHARMACY_SCORE']) /2
    data_all['ave_lif_tra_sc'] = (data_all['ave_lif_sc'] + data_all['ave_tra_sc'] ) /2
    data_all['ave_ent_tra_sc'] = (data_all['ave_ent_sc'] + data_all['ave_tra_sc'] ) /2
    data_all['ave_lif_ent_sc']= (data_all['ave_lif_sc'] + data_all['ave_ent_sc'] ) /2
    #### CNT 只有下列五個設施有統計
    data_all = prosperity(data_all,'PARK')
    data_all = prosperity(data_all,'STORE')
    data_all = prosperity(data_all,'CNVCSTORE')
    data_all = prosperity(data_all,'FASTFOOD')
    data_all = prosperity(data_all,'PHARMACY')
    data_all['ave_cnt'] = (data_all['PARK_CNT'] + data_all['STORE_CNT'] +data_all['CNVCSTORE_CNT']+data_all['FASTFOOD_CNT']+
                        data_all['PHARMACY_CNT']) / 5
    data_all['ROAD_YEARS'] = data_all['ROAD'].astype('str') + data_all['BUILD_YEARS'].astype('str')
    ### fill category na
    data_all['BUILD_USAGE'].replace(['住工用','停車空間','工商用','市場攤位'],'其他',inplace=True)
    data_all['BUILD_USAGE'].replace([np.nan],'未知用途',inplace=True)
    data_all['PARK_TYPE'].replace(np.nan,'無停車位',inplace=True)
    data_all['MRT_CROSS'].replace(np.nan,'無捷運',inplace=True)
    data_all['MRT_LINE'].replace(np.nan,'無捷運線',inplace=True)
    data_all['MRT_SQL'].replace(np.nan,'無捷運出口',inplace=True)
    data_all['HWAY_SQL'].replace(np.nan,'無交流道',inplace=True)
    data_all['CINEMA'].replace(np.nan,'無電影院',inplace=True)
    data_all['COMMUNITY_NAME'].replace(np.nan,'無社區',inplace=True)
    data_all['RAIL_SQL'].replace(np.nan,'無台鐵站',inplace=True)
    data_all['LAND_USAGE'].replace(np.nan,'未知用',inplace=True)
    data_all['BUILD_MATER'].replace(np.nan,'未知建材',inplace=True)
    
    #### drop else na and useless feature 
    data_all_na_col = []
    for col in data_all_col :
        if data_all[col].isna().sum() == 0 :
            pass
        else :
            data_all_na_col.append(col)
    #return data_all ,data_all_na_col
            
    data_all.drop(['BUILD_AGE','BUILD_DATE','BUILD_YEAR','CONSTRUCT_LOAN_BAL','CPC_MON_INC_RT','CPC_YR_INC_RT',
                          'CSM_LOAN_BAL','FIX_LOAN_BAL','HOUSE_LOAN_BAL','IGNORE_MEMO','IS_SC_FLAG','LHZ_LABEL',
                          'LICENSE_TYPE','PH.LABEL'],axis=1 , inplace = True)
    
    data_all.drop(['COMMUNITY_NO','VILLAGE_ID','SC.LABEL','REMARK','REGION_TYPE','ADDR',
                          ####  可用可不用  用的話要轉類別變數處裡
                          
                          ####  price 類的先全丟棄
                           'PRICE_ROAD_OUTLINER','PRICE_ROAD_3SIGMA_LOW','PRICE_ROAD_3SIGMA_HIGH','PRICE_HSYR_OUTLINER',
                          'PRICE_HSYR_3SIGMA_LOW','PRICE_HSYR_3SIGMA_HIGH','PRICE_BLDTP_OUTLINER','PRICE_BLDTP_3SIGMA_LOW',
                          'PRICE_BLDTP_3SIGMA_HIGH','MRT_LINE_CODE','price.road.sd','price.houseyear.sd','price.buildtype.sd',
                         'PRICE_OUTLINER','TRAD_YEARS',
                         ####  名稱變數選擇丟棄 ， 數量太多直接丟去 例如 : 便利商店名稱太雜亂 無用處
                         
                         'CNVCSTORE','VILLAGE_ID','FULL_VILLAGE'],axis=1,inplace=True)
    
    data_all.drop(['BUILD_T_PRICE','LAND_T_PRICE','LAND_U_PRICE','PARK_T_PRICE','T_PRICE','price.buildtype.cnt',
               'price.houseyear.cnt','price.road.cnt'],axis=1,inplace= True)
    

        
    #### nd  target encoding 
    nd_target_list_cate = []
    nd_onehot_list_cate = []
    for i in data_all.columns.tolist() :
        if data_all[i].dtype == 'object' :
            if len(data_all[i].unique()) >= 5 :
                nd_target_list_cate.append(i)
            else :
                nd_onehot_list_cate.append(i)
    
    nd_target_list_int = []
    nd_onehot_list_int = []
    for i in data_all.columns.tolist() :
        if data_all[i].dtype == 'int64' :
            if 'SCORE' in i :
                pass
            elif 'CNT' in i :
                pass
            else :
                if len(data_all[i].unique()) > 5 :
                    nd_target_list_int.append(i)
                else: 
                    nd_onehot_list_int.append(i)
    
    data_all.drop(['BUILD_T_PRICE.ADJ','BUILD_U_PRICE_ADJ_INDEX','CURRENT.PRICE.ADJ.RATE','PRICE.ADJUST.RATE',
              'U_PRICE','price.buildtype.mean','price.houseyear.mean','price.road.mean','sum.park.price'],axis=1,inplace=True)
    na_list = []
    
    
    
    print(data_all.shape)
    for col in data_all.columns.tolist() :
        if col != 'BUILD_U_PRICE.ADJ' :
            if data_all[col].isna().sum() > 1000 :
                na_list.append(col)
    data_all.drop(na_list,axis=1,inplace=True)
    print(data_all.shape)
    
    data_all.dropna(inplace = True)
    print(data_all.shape)
    
    data_all = get_one_hot(data_all,nd_onehot_list_cate)
    data_all = get_one_hot(data_all,nd_onehot_list_int)
    
    ################################## for xgboost 

    ###### 0.75 
    pos = int(len(data_all)*0.75)
        
    train = data_all.sample(frac = 1,random_state=1106).iloc[:pos,:]
    test = data_all.sample(frac = 1,random_state=1106).iloc[pos:,:]

    #### target encoding 
    train,test = get_target_encoding_cate(train,test,'BUILD_U_PRICE.ADJ',nd_target_list_cate)
    train,test = get_target_encoding_int(train,test,'BUILD_U_PRICE.ADJ',nd_target_list_int)
    
    
    
    
    xgb_model = xgb.XGBRegressor(n_estimators=600,objective = 'reg:squarederror',subsample=0.8,
                             max_depth=10,learning_rate=0.01,gamma=0.95,
                            colsample_bytree=0.99,reg_lambda=0.7,min_child_weight=0.9,
                             reg_alpha=0,n_jobs=5,verbosity=2,seed=1102)


    light_model = lgb.LGBMRegressor(boosting_type='gbdt',learning_rate=0.01,num_leaves=100,max_depth=15,subsample=0.88,
                                    colsample_bytree=0.4,reg_alpha=0.1,reg_lambda=0.4,n_estimators=600,n_jobs=5,seed=1102)


    cat_model = cat.CatBoostRegressor(n_estimators=600,one_hot_max_size=2, depth=8, l2_leaf_reg=3,
                                     random_seed=1102)


    forest_model = RandomForestRegressor(n_estimators=200,n_jobs=5,max_depth=12,random_state=1102)
    
    
    ##### for xgboost 
    
    
    ################################## for xgboost 
    for i in range(2) :
        cols = []
        for col in train.columns.tolist() :
            if ',' in col :
                cols.append(col.replace(',','-'))
            elif ']' in col :
                cols.append(col.replace(']',')'))
            else :
                cols.append(col)
        train.columns = cols

        ################################## for xgboost 

        colss = []
        for col in test.columns.tolist() :
            if ',' in col :
                colss.append(col.replace(',','-'))
            elif ']' in col :
                colss.append(col.replace(']',')'))
            else :
                colss.append(col)
        test.columns = colss
        
        
    print(train.shape)
    
    
    
    #xgb_mo = xgb_model.fit(train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1),train['BUILD_U_PRICE.ADJ'])
    lig_mo = light_model.fit(train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1),train['BUILD_U_PRICE.ADJ'])
    #cat_mo = cat_model.fit(train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1),train['BUILD_U_PRICE.ADJ'])
    #rf_mo = forest_model.fit(train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1),train['BUILD_U_PRICE.ADJ'])
    
    #f_cv = KFold(n_splits=5)
    
    #cv = cross_val_score(light_model,train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1),train['BUILD_U_PRICE.ADJ'],cv=f_cv,n_jobs=5)
    
    return lig_mo , train ,test ,cv
    

In [ ]:
def run_all_code (data_all) :

    
    def get_target_encoding_cate (df, test, target, nd_target_list):
        tar = TargetEncoder(smoothing=0.987).fit(df[nd_target_list], df[target])
        df[nd_target_list] = tar.transform(df[nd_target_list])
        test[nd_target_list] = tar.transform(test[nd_target_list])

        return df, test


    def get_target_encoding_int(df, test, target, nd_target_list):
        tar = TargetEncoder(smoothing=0.987).fit(df[nd_target_list].astype('str'), df[target])
        df[nd_target_list] = tar.transform(df[nd_target_list])
        test[nd_target_list] = tar.transform(test[nd_target_list])

        return df, test
    data_all_col = data_all.columns.tolist()
    data_all_na_col = []
    for col in data_all_col :
        if data_all[col].isna().sum() == 0 :
            pass
        else :
            data_all_na_col.append(col)
            
    ### fill na data
    data_all = fill_address_na(data_all,'ADDR')
    data_all = fill_address_na(data_all,'VILLAGE')
    data_all = fill_address_na(data_all,'OTROAD')
    data_all = new_address(data_all,'ADDR','ROAD')
    data_all = fill_all_disance(data_all,'BUILD_U_PRICE.ADJ',data_all_na_col)
    ### feature cross
    data_all = com_ornot(data_all,'COMMUNITY_NAME','IS_COMMUNITY')
    data_all = top_floor(data_all,'TRANS_FLOOR','TOTAL_FLOOR_NUM','TOP_FLOOR')
    data_all = get_month_build(data_all,'BUILD_DATE','BUILD_MONTH')
    data_all = get_month_trad(data_all,'TRADE_YM','TRAD_MONTH')
    data_all = get_years_build(data_all,'BUILD_DATE','BUILD_YEARS')
    data_all = get_years_trad(data_all,'TRADE_YM','TRAD_YEARS')
    data_all = cov_all_distance_log1p(data_all,data_all_col)
    data_all = cut_lat(data_all,'LATITUDE','lat_cut')
    data_all = cut_lat(data_all,'LONGITUDE','lon_cut')
    data_all = cross_col(data_all,'LATITUDE','LONGITUDE','cross_latlon')
    ### math tranlate
    data_all['LATITUDE.sqr'] = data_all['LATITUDE']**2
    data_all['LONGITUDE.sqr'] = data_all['LONGITUDE']**2
    data_all['HOUSE_YEAR.sqr'] = data_all['HOUSE_YEAR'] **2
    data_all['BUILD_AREA.sqr'] = data_all['BUILD_AREA'] **2
    data_all['TOTAL_FLOOR.sqr'] = data_all['TOTAL_FLOOR'] **2
    ### cip feature
    data_all['area_ratio'] = data_all['LAND_AREA']  / data_all['BUILD_AREA.ADJ']
    data_all['floor_ratio'] = data_all['TRANS_FLOOR']  / data_all['TOTAL_FLOOR_NUM']
    data_all['ave_tra_dis'] = (data_all['MRT_DISTANCE_SQL'] + data_all['HWAY_DISTANCE_SQL'] + data_all['RAIL_DISTANCE_SQL']) /3
    data_all['ave_ent_dis'] = (data_all['PARK_DISTANCE_SQL'] + data_all['CINEMA_DISTANCE']) /2
    data_all['ave_lif_dis'] = (data_all['MARKET_DISTANCE'] + data_all['STORE_DISTANCE_SQL'] + data_all['OILSTATION_DISTANCE']
                               + data_all['FASTFOOD_DISTANCE'] +data_all['CNVCSTORE_DISTANCE'] )/5 
    data_all['ave_health_dis'] = (data_all['HOSPITAL_DISTANCE_SQL'] + data_all['PHARMACY_DISTANCE']) /2
    data_all['ave_lif_tra_dis'] = (data_all['ave_lif_dis'] + data_all['ave_tra_dis'] ) /2
    data_all['ave_ent_tra_dis'] = (data_all['ave_ent_dis'] + data_all['ave_tra_dis'] ) / 2
    data_all['ave_lif_ent_dis']= (data_all['ave_lif_dis'] + data_all['ave_ent_dis'] ) /2
    data_all['ave_tra_sc'] = (data_all['MRT_SCORE'] + data_all['HWAY_SCORE'] + data_all['RAIL_SCORE']) /3
    data_all['ave_ent_sc'] = (data_all['PARK_SCORE'] + data_all['CINEMA_SCORE']) /2
    data_all['ave_lif_sc'] = (data_all['MARKET_SCORE'] + data_all['STORE_SCORE'] + data_all['FASTFOOD_SCORE'] + data_all['CNVCSTORE_SCORE']) /4 
    data_all['ave_health_sc'] = (data_all['HOSPITAL_SCORE'] + data_all['PHARMACY_SCORE']) /2
    data_all['ave_lif_tra_sc'] = (data_all['ave_lif_sc'] + data_all['ave_tra_sc'] ) /2
    data_all['ave_ent_tra_sc'] = (data_all['ave_ent_sc'] + data_all['ave_tra_sc'] ) /2
    data_all['ave_lif_ent_sc']= (data_all['ave_lif_sc'] + data_all['ave_ent_sc'] ) /2
    #### CNT 只有下列五個設施有統計
    data_all = prosperity(data_all,'PARK')
    data_all = prosperity(data_all,'STORE')
    data_all = prosperity(data_all,'CNVCSTORE')
    data_all = prosperity(data_all,'FASTFOOD')
    data_all = prosperity(data_all,'PHARMACY')
    data_all['ave_cnt'] = (data_all['PARK_CNT'] + data_all['STORE_CNT'] +data_all['CNVCSTORE_CNT']+data_all['FASTFOOD_CNT']+
                        data_all['PHARMACY_CNT']) / 5
    data_all['ROAD_YEARS'] = data_all['ROAD'].astype('str') + data_all['BUILD_YEARS'].astype('str')
    ### fill category na
    data_all['BUILD_USAGE'].replace(['住工用','停車空間','工商用','市場攤位'],'其他',inplace=True)
    data_all['BUILD_USAGE'].replace([np.nan],'未知用途',inplace=True)
    data_all['PARK_TYPE'].replace(np.nan,'無停車位',inplace=True)
    data_all['MRT_CROSS'].replace(np.nan,'無捷運',inplace=True)
    data_all['MRT_LINE'].replace(np.nan,'無捷運線',inplace=True)
    data_all['MRT_SQL'].replace(np.nan,'無捷運出口',inplace=True)
    data_all['HWAY_SQL'].replace(np.nan,'無交流道',inplace=True)
    data_all['CINEMA'].replace(np.nan,'無電影院',inplace=True)
    data_all['COMMUNITY_NAME'].replace(np.nan,'無社區',inplace=True)
    data_all['RAIL_SQL'].replace(np.nan,'無台鐵站',inplace=True)
    data_all['LAND_USAGE'].replace(np.nan,'未知用',inplace=True)
    data_all['BUILD_MATER'].replace(np.nan,'未知建材',inplace=True)
    
    #### drop else na and useless feature 
    data_all_na_col = []
    for col in data_all_col :
        if data_all[col].isna().sum() == 0 :
            pass
        else :
            data_all_na_col.append(col)
    #return data_all ,data_all_na_col
            
    data_all.drop(['BUILD_AGE','BUILD_DATE','BUILD_YEAR','CONSTRUCT_LOAN_BAL','CPC_MON_INC_RT','CPC_YR_INC_RT',
                          'CSM_LOAN_BAL','FIX_LOAN_BAL','HOUSE_LOAN_BAL','IGNORE_MEMO','IS_SC_FLAG','LHZ_LABEL',
                          'LICENSE_TYPE','PH.LABEL'],axis=1 , inplace = True)
    
    data_all.drop(['COMMUNITY_NO','VILLAGE_ID','SC.LABEL','REMARK','REGION_TYPE','ADDR',
                          ####  可用可不用  用的話要轉類別變數處裡
                          
                          ####  price 類的先全丟棄
                           'PRICE_ROAD_OUTLINER','PRICE_ROAD_3SIGMA_LOW','PRICE_ROAD_3SIGMA_HIGH','PRICE_HSYR_OUTLINER',
                          'PRICE_HSYR_3SIGMA_LOW','PRICE_HSYR_3SIGMA_HIGH','PRICE_BLDTP_OUTLINER','PRICE_BLDTP_3SIGMA_LOW',
                          'PRICE_BLDTP_3SIGMA_HIGH','MRT_LINE_CODE','price.road.sd','price.houseyear.sd','price.buildtype.sd',
                         'PRICE_OUTLINER','TRAD_YEARS',
                         ####  名稱變數選擇丟棄 ， 數量太多直接丟去 例如 : 便利商店名稱太雜亂 無用處
                         
                         'CNVCSTORE','VILLAGE_ID','FULL_VILLAGE'],axis=1,inplace=True)
    
    data_all.drop(['BUILD_T_PRICE','LAND_T_PRICE','LAND_U_PRICE','PARK_T_PRICE','T_PRICE','price.buildtype.cnt',
               'price.houseyear.cnt','price.road.cnt'],axis=1,inplace= True)
    

        
    #### nd  target encoding 
    nd_target_list_cate = []
    nd_onehot_list_cate = []
    for i in data_all.columns.tolist() :
        if data_all[i].dtype == 'object' :
            if len(data_all[i].unique()) >= 5 :
                nd_target_list_cate.append(i)
            else :
                nd_onehot_list_cate.append(i)
    
    nd_target_list_int = []
    nd_onehot_list_int = []
    for i in data_all.columns.tolist() :
        if data_all[i].dtype == 'int64' :
            if 'SCORE' in i :
                pass
            elif 'CNT' in i :
                pass
            else :
                if len(data_all[i].unique()) > 5 :
                    nd_target_list_int.append(i)
                else: 
                    nd_onehot_list_int.append(i)
    
    data_all.drop(['BUILD_T_PRICE.ADJ','BUILD_U_PRICE_ADJ_INDEX','CURRENT.PRICE.ADJ.RATE','PRICE.ADJUST.RATE',
              'U_PRICE','price.buildtype.mean','price.houseyear.mean','price.road.mean','sum.park.price'],axis=1,inplace=True)
    na_list = []
    
    
    
    print(data_all.shape)
    for col in data_all.columns.tolist() :
        if col != 'BUILD_U_PRICE.ADJ' :
            if data_all[col].isna().sum() > 1000 :
                na_list.append(col)
    data_all.drop(na_list,axis=1,inplace=True)
    print(data_all.shape)
    
    data_all.dropna(inplace = True)
    print(data_all.shape)
    
    data_all = get_one_hot(data_all,nd_onehot_list_cate)
    data_all = get_one_hot(data_all,nd_onehot_list_int)
    
    ################################## for xgboost 

    ###### 0.75 
    pos = int(len(data_all)*0.75)
        
    train = data_all.sample(frac = 1,random_state=1106).iloc[:pos,:]
    test = data_all.sample(frac = 1,random_state=1106).iloc[pos:,:]

    #### target encoding 
    train,test = get_target_encoding_cate(train,test,'BUILD_U_PRICE.ADJ',nd_target_list_cate)
    train,test = get_target_encoding_int(train,test,'BUILD_U_PRICE.ADJ',nd_target_list_int)
    
    
    
    
    xgb_model = xgb.XGBRegressor(n_estimators=600,objective = 'reg:squarederror',subsample=0.8,
                             max_depth=10,learning_rate=0.01,gamma=0.95,
                            colsample_bytree=0.99,reg_lambda=0.7,min_child_weight=0.9,
                             reg_alpha=0,n_jobs=5,verbosity=2,seed=1102)


    light_model = lgb.LGBMRegressor(boosting_type='gbdt',learning_rate=0.01,num_leaves=100,max_depth=15,subsample=0.88,
                                    colsample_bytree=0.4,reg_alpha=0.1,reg_lambda=0.4,n_estimators=600,n_jobs=5,seed=1102)


    cat_model = cat.CatBoostRegressor(n_estimators=600,one_hot_max_size=2, depth=8, l2_leaf_reg=3,
                                     random_seed=1102)


    forest_model = RandomForestRegressor(n_estimators=200,n_jobs=5,max_depth=12,random_state=1102)
    
    
    ##### for xgboost 
    
    
    ################################## for xgboost 
    for i in range(2) :
        cols = []
        for col in train.columns.tolist() :
            if ',' in col :
                cols.append(col.replace(',','-'))
            elif ']' in col :
                cols.append(col.replace(']',')'))
            else :
                cols.append(col)
        train.columns = cols

        ################################## for xgboost 

        colss = []
        for col in test.columns.tolist() :
            if ',' in col :
                colss.append(col.replace(',','-'))
            elif ']' in col :
                colss.append(col.replace(']',')'))
            else :
                colss.append(col)
        test.columns = colss
        
        
    print(train.shape)
    
    
    
    #xgb_mo = xgb_model.fit(train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1),train['BUILD_U_PRICE.ADJ'])
    lig_mo = light_model.fit(train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1),train['BUILD_U_PRICE.ADJ'])
    #cat_mo = cat_model.fit(train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1),train['BUILD_U_PRICE.ADJ'])
    #rf_mo = forest_model.fit(train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1),train['BUILD_U_PRICE.ADJ'])
    
    #f_cv = KFold(n_splits=5)
    
    #cv = cross_val_score(light_model,train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1),train['BUILD_U_PRICE.ADJ'],cv=f_cv,n_jobs=5)
    
    return lig_mo , train ,test ,cv
    

In [174]:
city_hsin = data_all.iloc[:len_hsin,:]
city_kao = data_all.iloc[len_hsin:len_hsin+len_kao,:]
city_newtaipei = data_all.iloc[len_hsin+len_kao:len_hsin+len_kao+len_newtaipei,:]
city_taichung = data_all.iloc[len_hsin+len_kao+len_newtaipei:len_hsin+len_kao+len_newtaipei+len_taichung,:]
city_tainan = data_all.iloc[len_hsin+len_kao+len_newtaipei+len_taichung:len_hsin+len_kao+len_newtaipei+len_taichung+len_tainan,:]
city_taipei = data_all.iloc[len_hsin+len_kao+len_newtaipei+len_taichung+len_tainan:len_hsin+len_kao+len_newtaipei+len_taichung+len_tainan+len_taipei,:]
city_taoyuan = data_all.iloc[len_hsin+len_kao+len_newtaipei+len_taichung+len_tainan+len_taipei:,:]

In [189]:
city_hsin_train = city_hsin[city_hsin['TRADE_YM'] <= 10700]
city_hsin_test = city_hsin[city_hsin['TRADE_YM'] > 10700]
city_kao_train = city_kao[city_kao['TRADE_YM'] <= 10700]
city_kao_test = city_kao[city_kao['TRADE_YM'] > 10700]
city_newtaipei_train = city_newtaipei[city_newtaipei['TRADE_YM'] <= 10700]
city_newtaipei_test = city_newtaipei[city_newtaipei['TRADE_YM'] > 10700]
city_taichung_train = city_taichung[city_taichung['TRADE_YM'] <= 10700]
city_taichung_test = city_taichung[city_taichung['TRADE_YM'] > 10700]
city_tainan_train = city_tainan[city_tainan['TRADE_YM'] <= 10700]
city_tainan_test = city_tainan[city_tainan['TRADE_YM'] > 10700]
city_taipei_train = city_taipei[city_taipei['TRADE_YM'] <= 10700]
city_taipei_test = city_taipei[city_taipei['TRADE_YM'] > 10700]
city_taoyuan_train = city_taoyuan[city_taoyuan['TRADE_YM'] <= 10700]
city_taoyuan_test = city_taoyuan[city_taoyuan['TRADE_YM'] > 10700]

In [190]:
every_city_train = [city_hsin_train,city_kao_train,city_newtaipei_train,city_taichung_train,city_tainan_train,city_taipei_train,city_taoyuan_train]
city_name = ['city_hsin','city_kao','city_newtaipei','city_taichung','city_tainan','city_taipei','city_taoyuan']

In [176]:
for city,name in zip(every_city,city_name) :
    
    lig ,train,test,cv= run_all_code(city)
    print(cv)
    print(name,'score=',lig.score(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1),test['BUILD_U_PRICE.ADJ']))
    print(name,'hit_rate',(((abs(test['BUILD_U_PRICE.ADJ'] - lig.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1)) ) / test['BUILD_U_PRICE.ADJ'])<=0.05).sum()) / len(test['BUILD_U_PRICE.ADJ']))
    col_index = train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1).columns
    col_importance = lig.feature_importances_
    col_import = pd.DataFrame(col_importance,index=col_index)
    print('{}_top30_feature'.format(name),col_import.sort_values(0,ascending=False)[:30])
    print('==================================================================================')

(43203, 154)
(43203, 150)
(43203, 150)
(32402, 193)
[0.8638269  0.86231865 0.8600656  0.86926847 0.8660656 ]
city_hsin score= 0.9090333548836049
city_hsin hit_rate 0.48486251273030273
city_hsin_top30_feature                        0
ROAD_YEARS          3073
cross_latlon        2828
BUILD_AREA.ADJ      2602
COMMUNITY_NAME      2305
area_ratio          2214
INDEX_ADJUST_RATE   1712
LAND_AREA           1679
BUILD_AREA          1642
TOTAL_BUILD_AREA    1412
OTROAD              1362
floor_ratio         1280
sum.park.area       1228
BUILD_YEARS         1173
TARGET_FLOOR        1033
ROAD                 997
HOUSE.INDEX          975
FASTFOOD_DISTANCE    844
BUILD_AREA.sqr       826
CINEMA_DISTANCE      816
VILLAGE              802
CNVCSTORE_DISTANCE   777
INPUT_DATE           690
LATITUDE             677
MARKET_DISTANCE      657
TRADE_VOL            636
TARGET_FLOOR_NUM     635
RAIL_DISTANCE_SQL    627
ave_ent_dis          623
PARK_U_PRICE         609
PHARMACY_DISTANCE    599
(87219, 154)
(872

In [254]:
print('{}_top30_feature'.format(name),col_import.sort_values(0,ascending=False)[:30])

city_taoyuan_top30_feature                           0
cross_latlon           3738
ROAD_YEARS             3392
COMMUNITY_NAME         2837
INDEX_ADJUST_RATE      2278
BUILD_AREA.ADJ         2158
area_ratio             1932
HOUSE.INDEX            1681
OTROAD                 1623
ROAD                   1453
LAND_AREA              1262
BUILD_YEARS            1125
sum.park.area          1121
BUILD_AREA             1114
TOTAL_BUILD_AREA       1014
INPUT_DATE              941
floor_ratio             912
VILLAGE                 883
CINEMA_DISTANCE         808
TARGET_FLOOR_NUM        700
SCHOOL_SQL              696
HOUSE_YEAR_RANGE        687
ave_lif_ent_dis         664
OILSTATION              657
BUILD_AREA.sqr          635
PHARMACY                611
TARGET_FLOOR            583
FASTFOOD_DISTANCE       567
FASTFOOD                563
LATITUDE                548
HOSPITAL_DISTANCE_SQL   548


In [163]:
lig.score(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1),test['BUILD_U_PRICE.ADJ'])

0.9088901126692985

In [164]:
f = lig.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
((((test['BUILD_U_PRICE.ADJ'] - f ) / test['BUILD_U_PRICE.ADJ'])<0.1).sum()) / len(test['BUILD_U_PRICE.ADJ'])

0.8734599913987199

In [165]:
col_index = train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1).columns
col_importance = lig.feature_importances_
col_import = pd.DataFrame(col_importance,index=col_index)

In [171]:
col_import.sort_values(0,ascending=False)[:30]

,0
COMMUNITY_NAME,3515
cross_latlon,2888
INDEX_ADJUST_RATE,2617
ROAD_YEARS,2561
BUILD_AREA.ADJ,1939
HOUSE.INDEX,1807
area_ratio,1374
BUILD_AREA,1323
OTROAD,1216
STORE_SQL,1128


In [154]:
def run_all_code_original (data_all) :

    data_all = data_all[['TOWN','LATITUDE','LONGITUDE','TRADE_DATE','TOTAL_FLOOR','BUILD_TYPE','BUILD_USAGE',
                         'HOUSE_YEAR','BUILD_AREA.ADJ','BUILD_U_PRICE.ADJ','PARK_T_PRICE','park.count','PARK_AREA',
                         'MRT_DISTANCE_SQL','PARK_DISTANCE_SQL','SCHOOL_DISTANCE_SQL','STORE_DISTANCE_SQL',
                         'HWAY_DISTANCE_SQL','RAIL_DISTANCE_SQL','HOSPITAL_DISTANCE_SQL','PHARMACY_DISTANCE',
                         'CINEMA_DISTANCE','CNVCSTORE_DISTANCE','FASTFOOD_DISTANCE','MARKET_DISTANCE','OILSTATION_DISTANCE',
                         'MRT_SCORE','PARK_SCORE','SCHOOL_SCORE','STORE_SCORE','HWAY_SCORE','RAIL_SCORE','HOSPITAL_SCORE',
                         'PHARMACY_SCORE','CINEMA_SCORE','CNVCSTORE_SCORE','FASTFOOD_SCORE','MARKET_SCORE','HAS_DISLIKE',
                        'DISLIKE_SCORE','TRAFFIC','LIVING','TRANS_FLOOR','TOTAL_FLOOR_NUM','BUILD_AREA']]
    
    data_all_col = data_all.columns.tolist()
    data_all_na_col = []
    for col in data_all_col :
        if data_all[col].isna().sum() == 0 :
            pass
        else :
            data_all_na_col.append(col)
    ### fill na data
    #data_all = fill_address_na(data_all,'ADDR')
    #data_all = fill_address_na(data_all,'VILLAGE')
    #data_all = fill_address_na(data_all,'OTROAD')
    #data_all = new_address(data_all,'ADDR','ROAD')
    #data_all = fill_all_disance(data_all,'BUILD_U_PRICE.ADJ',data_all_na_col)
    ### feature cross
    #data_all = com_ornot(data_all,'COMMUNITY_NAME','IS_COMMUNITY')
    data_all = top_floor(data_all,'TRANS_FLOOR','TOTAL_FLOOR_NUM','TOP_FLOOR')
    #data_all = get_month_build(data_all,'BUILD_DATE','BUILD_MONTH')
    #data_all = get_month_trad(data_all,'TRADE_YM','TRAD_MONTH')
    #data_all = get_years_build(data_all,'BUILD_DATE','BUILD_YEARS')
    #data_all = get_years_trad(data_all,'TRADE_YM','TRAD_YEARS')
    data_all = cov_all_distance_log1p(data_all,data_all_col)
    #data_all = cut_lat(data_all,'LATITUDE','lat_cut')
    #data_all = cut_lat(data_all,'LONGITUDE','lon_cut')
    #data_all = cross_col(data_all,'LATITUDE','LONGITUDE','cross_latlon')
    ### math tranlate
    data_all['LATITUDE.sqr'] = data_all['LATITUDE']**2
    data_all['LONGITUDE.sqr'] = data_all['LONGITUDE']**2
    data_all['HOUSE_YEAR.sqr'] = data_all['HOUSE_YEAR'] **2
    data_all['BUILD_AREA.sqr'] = data_all['BUILD_AREA'] **2
    data_all['TOTAL_FLOOR.sqr'] = data_all['TOTAL_FLOOR'] **2
    
    nd_onehot_list = []
    for i in data_all.columns.tolist() :
        if data_all[i].dtype == 'object' :
            nd_onehot_list.append(i)
            
    data_all = get_one_hot(data_all,nd_onehot_list)
    
    print(data_all.shape)
    
    na_list = []
    for col in data_all.columns.tolist() :
        if data_all[col].isna().sum() > 1000 :
            na_list.append(col)
    data_all.drop(na_list,axis=1,inplace=True)
    print(data_all.shape)
    
    data_all.dropna(inplace = True)
    print(data_all.shape)
    
    xgb_model = xgb.XGBRegressor()


    light_model = lgb.LGBMRegressor(n_estimators=100,n_jobs=5,seed=1102)


    cat_model = cat.CatBoostRegressor(n_estimators=600,one_hot_max_size=2, depth=8, l2_leaf_reg=3,
                                     random_seed=1102)


    forest_model = RandomForestRegressor(n_estimators=200,n_jobs=5,max_depth=12,random_state=1102)
    
    
    ##### for xgboost 
    
    X_train, X_test, y_train, y_test = train_test_split(data_all.drop('BUILD_U_PRICE.ADJ',axis=1), data_all['BUILD_U_PRICE.ADJ'], test_size=0.25, random_state=112)    
    
    ################################## for xgboost 
        
        
    print(X_train.shape)
    
    
    
    #xgb_mo = xgb_model.fit(X_train,y_train)
    lig_mo = light_model.fit(X_train,y_train)
    #cat_mo = cat_model.fit(train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1),train['BUILD_U_PRICE.ADJ'])
    #rf_mo = forest_model.fit(X_train,y_train)
    
    f_cv = KFold(n_splits=5)
    
    cv = cross_val_score(light_model,X_train,y_train,cv=f_cv,n_jobs=5)
    
    return lig_mo , X_test,y_test ,cv
    

In [159]:
city_hsin = data_all.iloc[:len_hsin,:]
city_kao = data_all.iloc[len_hsin:len_hsin+len_kao,:]
city_newtaipei = data_all.iloc[len_hsin+len_kao:len_hsin+len_kao+len_newtaipei,:]
city_taichung = data_all.iloc[len_hsin+len_kao+len_newtaipei:len_hsin+len_kao+len_newtaipei+len_taichung,:]
city_tainan = data_all.iloc[len_hsin+len_kao+len_newtaipei+len_taichung:len_hsin+len_kao+len_newtaipei+len_taichung+len_tainan,:]
city_taipei = data_all.iloc[len_hsin+len_kao+len_newtaipei+len_taichung+len_tainan:len_hsin+len_kao+len_newtaipei+len_taichung+len_tainan+len_taipei,:]
city_taoyuan = data_all.iloc[len_hsin+len_kao+len_newtaipei+len_taichung+len_tainan+len_taipei:,:]


In [160]:
every_city = [city_hsin,city_kao,city_newtaipei,city_taichung,city_tainan,city_taipei,city_taoyuan]
city_name = ['city_hsin','city_kao','city_newtaipei','city_taichung','city_tainan','city_taipei','city_taoyuan']

In [161]:
for city,name in zip(every_city,city_name) :
    lig ,tx,ty,cv= run_all_code_original(city)
    print(cv)
    print(name,'score=',lig.score(tx,ty))
    print(name,'hit_rate',(((((ty - lig.predict(tx)) ) / ty)<0.1).sum()) / len(ty))
    col_index = tx.columns
    col_importance = lig.feature_importances_
    col_import = pd.DataFrame(col_importance,index=col_index)
    print('{}_top30_feature'.format(name),col_import.sort_values(0,ascending=False)[:30])
    print('==================================================================================')

(43203, 73)
(43203, 72)
(42352, 72)
(31764, 71)
[0.86947097 0.867221   0.871873   0.86258171 0.86901905]
city_hsin score= 0.8756191869773076
city_hsin hit_rate 0.8593690970910465
city_hsin_top30_feature                            0
BUILD_AREA.ADJ           215
HOUSE_YEAR               192
TRADE_DATE               190
LATITUDE                 187
CINEMA_DISTANCE          178
LONGITUDE                142
STORE_DISTANCE_SQL       139
MARKET_DISTANCE          138
TOTAL_FLOOR              129
FASTFOOD_DISTANCE        126
HWAY_DISTANCE_SQL        124
SCHOOL_DISTANCE_SQL      113
RAIL_DISTANCE_SQL        111
BUILD_AREA               111
PARK_DISTANCE_SQL         97
HOSPITAL_DISTANCE_SQL     97
CNVCSTORE_DISTANCE        92
PHARMACY_DISTANCE         89
OILSTATION_DISTANCE       86
TRANS_FLOOR               61
PARK_AREA                 49
park.count                41
PARK_T_PRICE              41
BUILD_TYPE_套房(1房1廳1衛)     41
LIVING                    25
BUILD_USAGE_辦公室           24
DISLIKE_SCORE 

In [163]:
tx.columns.tolist()

['LATITUDE',
 'LONGITUDE',
 'TRADE_DATE',
 'TOTAL_FLOOR',
 'HOUSE_YEAR',
 'BUILD_AREA.ADJ',
 'PARK_T_PRICE',
 'park.count',
 'PARK_AREA',
 'PARK_DISTANCE_SQL',
 'SCHOOL_DISTANCE_SQL',
 'STORE_DISTANCE_SQL',
 'HWAY_DISTANCE_SQL',
 'RAIL_DISTANCE_SQL',
 'HOSPITAL_DISTANCE_SQL',
 'PHARMACY_DISTANCE',
 'CNVCSTORE_DISTANCE',
 'FASTFOOD_DISTANCE',
 'MARKET_DISTANCE',
 'OILSTATION_DISTANCE',
 'MRT_SCORE',
 'PARK_SCORE',
 'SCHOOL_SCORE',
 'STORE_SCORE',
 'HWAY_SCORE',
 'RAIL_SCORE',
 'HOSPITAL_SCORE',
 'PHARMACY_SCORE',
 'CINEMA_SCORE',
 'CNVCSTORE_SCORE',
 'FASTFOOD_SCORE',
 'MARKET_SCORE',
 'DISLIKE_SCORE',
 'TRAFFIC',
 'LIVING',
 'TRANS_FLOOR',
 'TOTAL_FLOOR_NUM',
 'BUILD_AREA',
 'TOP_FLOOR',
 'LATITUDE.sqr',
 'LONGITUDE.sqr',
 'HOUSE_YEAR.sqr',
 'BUILD_AREA.sqr',
 'TOTAL_FLOOR.sqr',
 'TOWN_中壢區',
 'TOWN_八德區',
 'TOWN_大園區',
 'TOWN_大溪區',
 'TOWN_平鎮區',
 'TOWN_新屋區',
 'TOWN_桃園區',
 'TOWN_楊梅區',
 'TOWN_蘆竹區',
 'TOWN_觀音區',
 'TOWN_龍潭區',
 'TOWN_龜山區',
 'BUILD_TYPE_住宅大樓(11層含以上有電梯)',
 'BUILD_TYPE_公寓(5樓含以下無電

In [12]:
def transform (data_all) :

    
    def get_target_encoding_cate (df, test, target, nd_target_list):
        tar = TargetEncoder(smoothing=0.987).fit(df[nd_target_list], df[target])
        df[nd_target_list] = tar.transform(df[nd_target_list])
        test[nd_target_list] = tar.transform(test[nd_target_list])

        return df, test


    def get_target_encoding_int(df, test, target, nd_target_list):
        tar = TargetEncoder(smoothing=0.987).fit(df[nd_target_list].astype('str'), df[target])
        df[nd_target_list] = tar.transform(df[nd_target_list])
        test[nd_target_list] = tar.transform(test[nd_target_list])

        return df, test
    data_all_col = data_all.columns.tolist()
    data_all_na_col = []
    for col in data_all_col :
        if data_all[col].isna().sum() == 0 :
            pass
        else :
            data_all_na_col.append(col)
            
    ### fill na data
    data_all = fill_address_na(data_all,'ADDR')
    data_all = fill_address_na(data_all,'VILLAGE')
    data_all = fill_address_na(data_all,'OTROAD')
    data_all = new_address(data_all,'ADDR','ROAD')
    data_all = fill_all_disance(data_all,'BUILD_U_PRICE.ADJ',data_all_na_col)
    ### feature cross
    data_all = com_ornot(data_all,'COMMUNITY_NAME','IS_COMMUNITY')
    data_all = top_floor(data_all,'TRANS_FLOOR','TOTAL_FLOOR_NUM','TOP_FLOOR')
    data_all = get_month_build(data_all,'BUILD_DATE','BUILD_MONTH')
    data_all = get_month_trad(data_all,'TRADE_YM','TRAD_MONTH')
    data_all = get_years_build(data_all,'BUILD_DATE','BUILD_YEARS')
    data_all = get_years_trad(data_all,'TRADE_YM','TRAD_YEARS')
    data_all = cov_all_distance_log1p(data_all,data_all_col)
    data_all = cut_lat(data_all,'LATITUDE','lat_cut')
    data_all = cut_lat(data_all,'LONGITUDE','lon_cut')
    data_all = cross_col(data_all,'LATITUDE','LONGITUDE','cross_latlon')
    ### math tranlate
    data_all['LATITUDE.sqr'] = data_all['LATITUDE']**2
    data_all['LONGITUDE.sqr'] = data_all['LONGITUDE']**2
    data_all['HOUSE_YEAR.sqr'] = data_all['HOUSE_YEAR'] **2
    data_all['BUILD_AREA.sqr'] = data_all['BUILD_AREA'] **2
    data_all['TOTAL_FLOOR.sqr'] = data_all['TOTAL_FLOOR'] **2
    ### cip feature
    data_all['area_ratio'] = data_all['LAND_AREA']  / data_all['BUILD_AREA.ADJ']
    data_all['floor_ratio'] = data_all['TRANS_FLOOR']  / data_all['TOTAL_FLOOR_NUM']
    data_all['ave_tra_dis'] = (data_all['MRT_DISTANCE_SQL'] + data_all['HWAY_DISTANCE_SQL'] + data_all['RAIL_DISTANCE_SQL']) /3
    data_all['ave_ent_dis'] = (data_all['PARK_DISTANCE_SQL'] + data_all['CINEMA_DISTANCE']) /2
    data_all['ave_lif_dis'] = (data_all['MARKET_DISTANCE'] + data_all['STORE_DISTANCE_SQL'] + data_all['OILSTATION_DISTANCE']
                               + data_all['FASTFOOD_DISTANCE'] +data_all['CNVCSTORE_DISTANCE'] )/5 
    data_all['ave_health_dis'] = (data_all['HOSPITAL_DISTANCE_SQL'] + data_all['PHARMACY_DISTANCE']) /2
    data_all['ave_lif_tra_dis'] = (data_all['ave_lif_dis'] + data_all['ave_tra_dis'] ) /2
    data_all['ave_ent_tra_dis'] = (data_all['ave_ent_dis'] + data_all['ave_tra_dis'] ) / 2
    data_all['ave_lif_ent_dis']= (data_all['ave_lif_dis'] + data_all['ave_ent_dis'] ) /2
    data_all['ave_tra_sc'] = (data_all['MRT_SCORE'] + data_all['HWAY_SCORE'] + data_all['RAIL_SCORE']) /3
    data_all['ave_ent_sc'] = (data_all['PARK_SCORE'] + data_all['CINEMA_SCORE']) /2
    data_all['ave_lif_sc'] = (data_all['MARKET_SCORE'] + data_all['STORE_SCORE'] + data_all['FASTFOOD_SCORE'] + data_all['CNVCSTORE_SCORE']) /4 
    data_all['ave_health_sc'] = (data_all['HOSPITAL_SCORE'] + data_all['PHARMACY_SCORE']) /2
    data_all['ave_lif_tra_sc'] = (data_all['ave_lif_sc'] + data_all['ave_tra_sc'] ) /2
    data_all['ave_ent_tra_sc'] = (data_all['ave_ent_sc'] + data_all['ave_tra_sc'] ) /2
    data_all['ave_lif_ent_sc']= (data_all['ave_lif_sc'] + data_all['ave_ent_sc'] ) /2
    #### CNT 只有下列五個設施有統計
    data_all = prosperity(data_all,'PARK')
    data_all = prosperity(data_all,'STORE')
    data_all = prosperity(data_all,'CNVCSTORE')
    data_all = prosperity(data_all,'FASTFOOD')
    data_all = prosperity(data_all,'PHARMACY')
    data_all['ave_cnt'] = (data_all['PARK_CNT'] + data_all['STORE_CNT'] +data_all['CNVCSTORE_CNT']+data_all['FASTFOOD_CNT']+
                        data_all['PHARMACY_CNT']) / 5
    data_all['ROAD_YEARS'] = data_all['ROAD'].astype('str') + data_all['BUILD_YEARS'].astype('str')
    ### fill category na
    data_all['BUILD_USAGE'].replace(['住工用','停車空間','工商用','市場攤位'],'其他',inplace=True)
    data_all['BUILD_USAGE'].replace([np.nan],'未知用途',inplace=True)
    data_all['PARK_TYPE'].replace(np.nan,'無停車位',inplace=True)
    data_all['MRT_CROSS'].replace(np.nan,'無捷運',inplace=True)
    data_all['MRT_LINE'].replace(np.nan,'無捷運線',inplace=True)
    data_all['MRT_SQL'].replace(np.nan,'無捷運出口',inplace=True)
    data_all['HWAY_SQL'].replace(np.nan,'無交流道',inplace=True)
    data_all['CINEMA'].replace(np.nan,'無電影院',inplace=True)
    data_all['COMMUNITY_NAME'].replace(np.nan,'無社區',inplace=True)
    data_all['RAIL_SQL'].replace(np.nan,'無台鐵站',inplace=True)
    data_all['LAND_USAGE'].replace(np.nan,'未知用',inplace=True)
    data_all['BUILD_MATER'].replace(np.nan,'未知建材',inplace=True)
    
    #### drop else na and useless feature 
    data_all_na_col = []
    for col in data_all_col :
        if data_all[col].isna().sum() == 0 :
            pass
        else :
            data_all_na_col.append(col)
    #return data_all ,data_all_na_col
            
    data_all.drop(['BUILD_AGE','BUILD_DATE','BUILD_YEAR','CONSTRUCT_LOAN_BAL','CPC_MON_INC_RT','CPC_YR_INC_RT',
                          'CSM_LOAN_BAL','FIX_LOAN_BAL','HOUSE_LOAN_BAL','IGNORE_MEMO','IS_SC_FLAG','LHZ_LABEL',
                          'LICENSE_TYPE','PH.LABEL'],axis=1 , inplace = True)
    
    data_all.drop(['COMMUNITY_NO','VILLAGE_ID','SC.LABEL','REMARK','REGION_TYPE','ADDR',
                          ####  可用可不用  用的話要轉類別變數處裡
                          
                          ####  price 類的先全丟棄
                           'PRICE_ROAD_OUTLINER','PRICE_ROAD_3SIGMA_LOW','PRICE_ROAD_3SIGMA_HIGH','PRICE_HSYR_OUTLINER',
                          'PRICE_HSYR_3SIGMA_LOW','PRICE_HSYR_3SIGMA_HIGH','PRICE_BLDTP_OUTLINER','PRICE_BLDTP_3SIGMA_LOW',
                          'PRICE_BLDTP_3SIGMA_HIGH','MRT_LINE_CODE','price.road.sd','price.houseyear.sd','price.buildtype.sd',
                         'PRICE_OUTLINER','TRAD_YEARS',
                         ####  名稱變數選擇丟棄 ， 數量太多直接丟去 例如 : 便利商店名稱太雜亂 無用處
                         
                         'CNVCSTORE','VILLAGE_ID','FULL_VILLAGE'],axis=1,inplace=True)
    
    data_all.drop(['BUILD_T_PRICE','LAND_T_PRICE','LAND_U_PRICE','PARK_T_PRICE','T_PRICE','price.buildtype.cnt',
               'price.houseyear.cnt','price.road.cnt'],axis=1,inplace= True)
    

        
    #### nd  target encoding 
    nd_target_list_cate = []
    nd_onehot_list_cate = []
    for i in data_all.columns.tolist() :
        if data_all[i].dtype == 'object' :
            if len(data_all[i].unique()) >= 5 :
                nd_target_list_cate.append(i)
            else :
                nd_onehot_list_cate.append(i)
    
    nd_target_list_int = []
    nd_onehot_list_int = []
    for i in data_all.columns.tolist() :
        if data_all[i].dtype == 'int64' :
            if 'SCORE' in i :
                pass
            elif 'CNT' in i :
                pass
            else :
                if len(data_all[i].unique()) > 5 :
                    nd_target_list_int.append(i)
                else: 
                    nd_onehot_list_int.append(i)
    
    data_all.drop(['BUILD_T_PRICE.ADJ','BUILD_U_PRICE_ADJ_INDEX','CURRENT.PRICE.ADJ.RATE','PRICE.ADJUST.RATE',
              'U_PRICE','price.buildtype.mean','price.houseyear.mean','price.road.mean','sum.park.price'],axis=1,inplace=True)

    na_list = []
    
    print(data_all.shape)
    for col in data_all.columns.tolist() :
        if col != 'BUILD_U_PRICE.ADJ' :
            if data_all[col].isna().sum() > 1000 :
                na_list.append(col)
    data_all.drop(na_list,axis=1,inplace=True)
    print(data_all.shape)

    
    data_all = get_one_hot(data_all,nd_onehot_list_cate)
    data_all = get_one_hot(data_all,nd_onehot_list_int)
    
    train = data_all.dropna()
    test = data_all[data_all['BUILD_U_PRICE.ADJ'].isna()]
    
    train,test = get_target_encoding_cate(train,test,'BUILD_U_PRICE.ADJ',nd_target_list_cate)
    train,test = get_target_encoding_int(train,test,'BUILD_U_PRICE.ADJ',nd_target_list_int)
    
    return train,test
    

In [15]:
train,test = transform(data_all_kaggle)

(573415, 155)
(573415, 155)


In [16]:
test.shape

(191237, 203)

In [17]:
data_all_kaggle.shape

(573415, 155)

In [38]:
xgb_model = xgb.XGBRegressor()


light_model = lgb.LGBMRegressor(boosting_type='gbdt',learning_rate=0.01,num_leaves=100,max_depth=15,subsample=0.88,
                                    colsample_bytree=0.4,reg_alpha=0.1,reg_lambda=0.4,n_estimators=600,n_jobs=5)


cat_model = cat.CatBoostRegressor(n_estimators=600,one_hot_max_size=2, depth=8, l2_leaf_reg=3,
                                 random_seed=1102)


forest_model = RandomForestRegressor(n_estimators=200,n_jobs=5,max_depth=12,random_state=1102)


##### for xgboost 

X_train, X_test, y_train, y_test = train_test_split(train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1), train['BUILD_U_PRICE.ADJ'], test_size=0.15, random_state=112)    

################################## for xgboost 


print(X_train.shape)



#xgb_mo = xgb_model.fit(X_train,y_train)
lig_mo = light_model.fit(X_train,y_train)
#cat_mo = cat_model.fit(train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1),train['BUILD_U_PRICE.ADJ'])
#rf_mo = forest_model.fit(X_train,y_train)

f_cv = KFold(n_splits=5)

cv = cross_val_score(light_model,X_train,y_train,cv=f_cv,n_jobs=5)


(324805, 201)


In [40]:
cv

array([0.96391339, 0.96394914, 0.96394311, 0.9635837 , 0.96386957])

In [41]:
pre = lig_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))

In [42]:
((((test_set_ans - pre ) / test_set_ans ) < 0.1 ).sum() ) / len(test_set_ans)

0.7392136458948844

In [142]:
def run_all_code_original_kaggle (data_all) :
 
    data_all = data_all[['TOWN','LATITUDE','LONGITUDE','TRADE_DATE','TOTAL_FLOOR','BUILD_TYPE','BUILD_USAGE',
                         'HOUSE_YEAR','BUILD_AREA.ADJ','BUILD_U_PRICE.ADJ','PARK_T_PRICE','park.count','PARK_AREA',
                         'MRT_DISTANCE_SQL','PARK_DISTANCE_SQL','SCHOOL_DISTANCE_SQL','STORE_DISTANCE_SQL',
                         'HWAY_DISTANCE_SQL','RAIL_DISTANCE_SQL','HOSPITAL_DISTANCE_SQL','PHARMACY_DISTANCE',
                         'CINEMA_DISTANCE','CNVCSTORE_DISTANCE','FASTFOOD_DISTANCE','MARKET_DISTANCE','OILSTATION_DISTANCE',
                         'MRT_SCORE','PARK_SCORE','SCHOOL_SCORE','STORE_SCORE','HWAY_SCORE','RAIL_SCORE','HOSPITAL_SCORE',
                         'PHARMACY_SCORE','CINEMA_SCORE','CNVCSTORE_SCORE','FASTFOOD_SCORE','MARKET_SCORE','HAS_DISLIKE',
                        'DISLIKE_SCORE','TRAFFIC','LIVING','TRANS_FLOOR','TOTAL_FLOOR_NUM','BUILD_AREA']]
    
    data_all_col = data_all.columns.tolist()
    data_all_na_col = []
    for col in data_all_col :
        if data_all[col].isna().sum() == 0 :
            pass
        else :
            data_all_na_col.append(col)
    ### fill na data
    #data_all = fill_address_na(data_all,'ADDR')
    #data_all = fill_address_na(data_all,'VILLAGE')
    #data_all = fill_address_na(data_all,'OTROAD')
    #data_all = new_address(data_all,'ADDR','ROAD')
    #data_all = fill_all_disance(data_all,'BUILD_U_PRICE.ADJ',data_all_na_col)
    ### feature cross
    #data_all = com_ornot(data_all,'COMMUNITY_NAME','IS_COMMUNITY')
    data_all = top_floor(data_all,'TRANS_FLOOR','TOTAL_FLOOR_NUM','TOP_FLOOR')
    #data_all = get_month_build(data_all,'BUILD_DATE','BUILD_MONTH')
    #data_all = get_month_trad(data_all,'TRADE_YM','TRAD_MONTH')
    #data_all = get_years_build(data_all,'BUILD_DATE','BUILD_YEARS')
    #data_all = get_years_trad(data_all,'TRADE_YM','TRAD_YEARS')
    data_all = cov_all_distance_log1p(data_all,data_all_col)
    #data_all = cut_lat(data_all,'LATITUDE','lat_cut')
    #data_all = cut_lat(data_all,'LONGITUDE','lon_cut')
    #data_all = cross_col(data_all,'LATITUDE','LONGITUDE','cross_latlon')
    ### math tranlate
    data_all['LATITUDE.sqr'] = data_all['LATITUDE']**2
    data_all['LONGITUDE.sqr'] = data_all['LONGITUDE']**2
    data_all['HOUSE_YEAR.sqr'] = data_all['HOUSE_YEAR'] **2
    data_all['BUILD_AREA.sqr'] = data_all['BUILD_AREA'] **2
    data_all['TOTAL_FLOOR.sqr'] = data_all['TOTAL_FLOOR'] **2
    print(data_all.shape)
    nd_onehot_list = []
    for i in data_all.columns.tolist() :
        if data_all[i].dtype == 'object' :
            nd_onehot_list.append(i)
            
    data_all = get_one_hot(data_all,nd_onehot_list)
    
    print(data_all.shape)
    
    na_list = []
    
    print(data_all.shape)
    for col in data_all.columns.tolist() :
        if col != 'BUILD_U_PRICE.ADJ' :
            if data_all[col].isna().sum() > 1000 :
                na_list.append(col)
    data_all.drop(na_list,axis=1,inplace=True)
    
    train = data_all.dropna()
    test = data_all[data_all['BUILD_U_PRICE.ADJ'].isna()]
    
    
    print(train.shape)
    
    xgb_model = xgb.XGBRegressor()


    light_model = lgb.LGBMRegressor(n_estimators=100,n_jobs=5)


    cat_model = cat.CatBoostRegressor(n_estimators=600,one_hot_max_size=2, depth=8, l2_leaf_reg=3,
                                     random_seed=1102)


    forest_model = RandomForestRegressor(n_estimators=200,n_jobs=5,max_depth=12,random_state=1102)
    
    
    ##### for xgboost 
    
    X_train, X_test, y_train, y_test = train_test_split(train.drop('BUILD_U_PRICE.ADJ',axis=1), train['BUILD_U_PRICE.ADJ'], test_size=0.15, random_state=112)    
    
    ################################## for xgboost 
        
        
    print(X_train.shape)
    
    
    
    xgb_mo = xgb_model.fit(X_train,y_train)
    #lig_mo = light_model.fit(X_train,y_train)
    #cat_mo = cat_model.fit(train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1),train['BUILD_U_PRICE.ADJ'])
    #rf_mo = forest_model.fit(X_train,y_train)
    
    f_cv = KFold(n_splits=5)
    
    #cv = cross_val_score(xgb_mo,X_train,y_train,cv=f_cv,n_jobs=5)
    
    
    
    
    return  xgb_mo  , test , train

In [143]:
xgb_mo , test_for_pre ,train_data= run_all_code_original_kaggle(data_all_kaggle)

(573415, 51)
(573415, 207)
(573415, 207)
(382124, 203)
(324805, 202)
[14:30:43] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [145]:
cccv

array([0.93999211, 0.94045109, 0.93961857, 0.93889901, 0.94059208])

In [146]:
pre = xg_mo.predict(test_for_pre.drop('BUILD_U_PRICE.ADJ',axis=1))

In [147]:
abs((((test_set_ans - pre ) / test_set_ans ) < 0.1 ).sum() ) / len(test_set_ans)

0.7289541249862735

In [148]:
col_index = train_data.drop('BUILD_U_PRICE.ADJ',axis=1).columns
col_importance = xgb_mo.feature_importances_
col_import = pd.DataFrame(col_importance,index=col_index)
print(col_import.sort_values(0,ascending=False)[:30])

                                0
LONGITUDE.sqr            0.230701
LONGITUDE                0.135972
MARKET_DISTANCE          0.123538
TRAFFIC                  0.078504
MRT_SCORE                0.065355
TOWN_大安區                 0.045172
BUILD_TYPE_公寓(5樓含以下無電梯)  0.037192
CINEMA_SCORE             0.035519
FASTFOOD_DISTANCE        0.029642
HOUSE_YEAR               0.025510
HWAY_SCORE               0.017793
LATITUDE                 0.016687
TOWN_板橋區                 0.013804
TOWN_信義區                 0.012076
TOWN_中正區                 0.011514
park.count               0.010453
BUILD_AREA               0.007799
PARK_AREA                0.007295
PARK_DISTANCE_SQL        0.006917
TOTAL_FLOOR              0.006752
PHARMACY_DISTANCE        0.005361
TOWN_松山區                 0.005081
LIVING                   0.004964
TRANS_FLOOR              0.004896
PARK_T_PRICE             0.004438
TRADE_DATE               0.004412
TOWN_中山區                 0.003946
LATITUDE.sqr             0.003834
HOSPITAL_DISTA

In [233]:
def transform (data_all) :

    
    def get_target_encoding_cate (df, test, target, nd_target_list):
        tar = TargetEncoder(smoothing=0.987).fit(df[nd_target_list], df[target])
        df[nd_target_list] = tar.transform(df[nd_target_list])
        test[nd_target_list] = tar.transform(test[nd_target_list])

        return df, test


    def get_target_encoding_int(df, test, target, nd_target_list):
        tar = TargetEncoder(smoothing=0.987).fit(df[nd_target_list].astype('str'), df[target])
        df[nd_target_list] = tar.transform(df[nd_target_list])
        test[nd_target_list] = tar.transform(test[nd_target_list])

        return df, test
    data_all_col = data_all.columns.tolist()
    data_all_na_col = []
    for col in data_all_col :
        if data_all[col].isna().sum() == 0 :
            pass
        else :
            data_all_na_col.append(col)
            
    ### fill na data
    data_all = fill_address_na(data_all,'ADDR')
    data_all = fill_address_na(data_all,'VILLAGE')
    data_all = fill_address_na(data_all,'OTROAD')
    data_all = new_address(data_all,'ADDR','ROAD')
    data_all = fill_all_disance(data_all,'BUILD_U_PRICE.ADJ',data_all_na_col)
    ### feature cross
    data_all = com_ornot(data_all,'COMMUNITY_NAME','IS_COMMUNITY')
    data_all = top_floor(data_all,'TRANS_FLOOR','TOTAL_FLOOR_NUM','TOP_FLOOR')
    data_all = get_month_build(data_all,'BUILD_DATE','BUILD_MONTH')
    data_all = get_month_trad(data_all,'TRADE_YM','TRAD_MONTH')
    data_all = get_years_build(data_all,'BUILD_DATE','BUILD_YEARS')
    data_all = get_years_trad(data_all,'TRADE_YM','TRAD_YEARS')
    data_all = cov_all_distance_log1p(data_all,data_all_col)
    data_all = cut_lat(data_all,'LATITUDE','lat_cut')
    data_all = cut_lat(data_all,'LONGITUDE','lon_cut')
    data_all = cross_col(data_all,'LATITUDE','LONGITUDE','cross_latlon')
    ### math tranlate
    data_all['LATITUDE.sqr'] = data_all['LATITUDE']**2
    data_all['LONGITUDE.sqr'] = data_all['LONGITUDE']**2
    data_all['HOUSE_YEAR.sqr'] = data_all['HOUSE_YEAR'] **2
    data_all['BUILD_AREA.sqr'] = data_all['BUILD_AREA'] **2
    data_all['TOTAL_FLOOR.sqr'] = data_all['TOTAL_FLOOR'] **2
    ### cip feature
    data_all['area_ratio'] = data_all['LAND_AREA']  / data_all['BUILD_AREA.ADJ']
    data_all['floor_ratio'] = data_all['TRANS_FLOOR']  / data_all['TOTAL_FLOOR_NUM']
    data_all['ave_tra_dis'] = (data_all['MRT_DISTANCE_SQL'] + data_all['HWAY_DISTANCE_SQL'] + data_all['RAIL_DISTANCE_SQL']) /3
    data_all['ave_ent_dis'] = (data_all['PARK_DISTANCE_SQL'] + data_all['CINEMA_DISTANCE']) /2
    data_all['ave_lif_dis'] = (data_all['MARKET_DISTANCE'] + data_all['STORE_DISTANCE_SQL'] + data_all['OILSTATION_DISTANCE']
                               + data_all['FASTFOOD_DISTANCE'] +data_all['CNVCSTORE_DISTANCE'] )/5 
    data_all['ave_health_dis'] = (data_all['HOSPITAL_DISTANCE_SQL'] + data_all['PHARMACY_DISTANCE']) /2
    data_all['ave_lif_tra_dis'] = (data_all['ave_lif_dis'] + data_all['ave_tra_dis'] ) /2
    data_all['ave_ent_tra_dis'] = (data_all['ave_ent_dis'] + data_all['ave_tra_dis'] ) / 2
    data_all['ave_lif_ent_dis']= (data_all['ave_lif_dis'] + data_all['ave_ent_dis'] ) /2
    data_all['ave_tra_sc'] = (data_all['MRT_SCORE'] + data_all['HWAY_SCORE'] + data_all['RAIL_SCORE']) /3
    data_all['ave_ent_sc'] = (data_all['PARK_SCORE'] + data_all['CINEMA_SCORE']) /2
    data_all['ave_lif_sc'] = (data_all['MARKET_SCORE'] + data_all['STORE_SCORE'] + data_all['FASTFOOD_SCORE'] + data_all['CNVCSTORE_SCORE']) /4 
    data_all['ave_health_sc'] = (data_all['HOSPITAL_SCORE'] + data_all['PHARMACY_SCORE']) /2
    data_all['ave_lif_tra_sc'] = (data_all['ave_lif_sc'] + data_all['ave_tra_sc'] ) /2
    data_all['ave_ent_tra_sc'] = (data_all['ave_ent_sc'] + data_all['ave_tra_sc'] ) /2
    data_all['ave_lif_ent_sc']= (data_all['ave_lif_sc'] + data_all['ave_ent_sc'] ) /2
    #### CNT 只有下列五個設施有統計
    data_all = prosperity(data_all,'PARK')
    data_all = prosperity(data_all,'STORE')
    data_all = prosperity(data_all,'CNVCSTORE')
    data_all = prosperity(data_all,'FASTFOOD')
    data_all = prosperity(data_all,'PHARMACY')
    data_all['ave_cnt'] = (data_all['PARK_CNT'] + data_all['STORE_CNT'] +data_all['CNVCSTORE_CNT']+data_all['FASTFOOD_CNT']+
                        data_all['PHARMACY_CNT']) / 5
    data_all['ROAD_YEARS'] = data_all['ROAD'].astype('str') + data_all['BUILD_YEARS'].astype('str')
    ### fill category na
    data_all['BUILD_USAGE'].replace(['住工用','停車空間','工商用','市場攤位'],'其他',inplace=True)
    data_all['BUILD_USAGE'].replace([np.nan],'未知用途',inplace=True)
    data_all['PARK_TYPE'].replace(np.nan,'無停車位',inplace=True)
    data_all['MRT_CROSS'].replace(np.nan,'無捷運',inplace=True)
    data_all['MRT_LINE'].replace(np.nan,'無捷運線',inplace=True)
    data_all['MRT_SQL'].replace(np.nan,'無捷運出口',inplace=True)
    data_all['HWAY_SQL'].replace(np.nan,'無交流道',inplace=True)
    data_all['CINEMA'].replace(np.nan,'無電影院',inplace=True)
    data_all['COMMUNITY_NAME'].replace(np.nan,'無社區',inplace=True)
    data_all['RAIL_SQL'].replace(np.nan,'無台鐵站',inplace=True)
    data_all['LAND_USAGE'].replace(np.nan,'未知用',inplace=True)
    data_all['BUILD_MATER'].replace(np.nan,'未知建材',inplace=True)
    
    #### drop else na and useless feature 
    data_all_na_col = []
    for col in data_all_col :
        if data_all[col].isna().sum() == 0 :
            pass
        else :
            data_all_na_col.append(col)
    #return data_all ,data_all_na_col
            
    data_all.drop(['BUILD_AGE','BUILD_DATE','BUILD_YEAR','CONSTRUCT_LOAN_BAL','CPC_MON_INC_RT','CPC_YR_INC_RT',
                          'CSM_LOAN_BAL','FIX_LOAN_BAL','HOUSE_LOAN_BAL','IGNORE_MEMO','IS_SC_FLAG','LHZ_LABEL',
                          'LICENSE_TYPE','PH.LABEL'],axis=1 , inplace = True)
    
    data_all.drop(['COMMUNITY_NO','VILLAGE_ID','SC.LABEL','REMARK','REGION_TYPE','ADDR',
                          ####  可用可不用  用的話要轉類別變數處裡
                          
                          ####  price 類的先全丟棄
                           'PRICE_ROAD_OUTLINER','PRICE_ROAD_3SIGMA_LOW','PRICE_ROAD_3SIGMA_HIGH','PRICE_HSYR_OUTLINER',
                          'PRICE_HSYR_3SIGMA_LOW','PRICE_HSYR_3SIGMA_HIGH','PRICE_BLDTP_OUTLINER','PRICE_BLDTP_3SIGMA_LOW',
                          'PRICE_BLDTP_3SIGMA_HIGH','MRT_LINE_CODE','price.road.sd','price.houseyear.sd','price.buildtype.sd',
                         'PRICE_OUTLINER','TRAD_YEARS',
                         ####  名稱變數選擇丟棄 ， 數量太多直接丟去 例如 : 便利商店名稱太雜亂 無用處
                         
                         'CNVCSTORE','VILLAGE_ID','FULL_VILLAGE'],axis=1,inplace=True)
    
    data_all.drop(['BUILD_T_PRICE','LAND_T_PRICE','LAND_U_PRICE','PARK_T_PRICE','T_PRICE','price.buildtype.cnt',
               'price.houseyear.cnt','price.road.cnt'],axis=1,inplace= True)
    

        
    #### nd  target encoding 
    nd_target_list_cate = []
    nd_onehot_list_cate = []
    for i in data_all.columns.tolist() :
        if data_all[i].dtype == 'object' :
            if len(data_all[i].unique()) >= 5 :
                nd_target_list_cate.append(i)
            else :
                nd_onehot_list_cate.append(i)
    
    nd_target_list_int = []
    nd_onehot_list_int = []
    for i in data_all.columns.tolist() :
        if data_all[i].dtype == 'int64' :
            if 'SCORE' in i :
                pass
            elif 'CNT' in i :
                pass
            else :
                if len(data_all[i].unique()) > 5 :
                    nd_target_list_int.append(i)
                else: 
                    nd_onehot_list_int.append(i)
    
    data_all.drop(['BUILD_T_PRICE.ADJ','BUILD_U_PRICE_ADJ_INDEX','CURRENT.PRICE.ADJ.RATE','PRICE.ADJUST.RATE',
              'U_PRICE','price.buildtype.mean','price.houseyear.mean','price.road.mean','sum.park.price'],axis=1,inplace=True)

    na_list = []
    
    print(data_all.shape)
    for col in data_all.columns.tolist() :
        if col != 'BUILD_U_PRICE.ADJ' :
            if data_all[col].isna().sum() > 1000 :
                na_list.append(col)
    data_all.drop(na_list,axis=1,inplace=True)
    print(data_all.shape)

    
    data_all = get_one_hot(data_all,nd_onehot_list_cate)
    data_all = get_one_hot(data_all,nd_onehot_list_int)
    
    train = data_all.dropna()
    test = data_all[data_all['BUILD_U_PRICE.ADJ'].isna()]
    
    train,test = get_target_encoding_cate(train,test,'BUILD_U_PRICE.ADJ',nd_target_list_cate)
    train,test = get_target_encoding_int(train,test,'BUILD_U_PRICE.ADJ',nd_target_list_int)
    
    for i in range(2) :
        cols = []
        for col in train.columns.tolist() :
            if ',' in col :
                cols.append(col.replace(',','-'))
            elif ']' in col :
                cols.append(col.replace(']',')'))
            else :
                cols.append(col)
        train.columns = cols

        ################################## for xgboost 

        colss = []
        for col in test.columns.tolist() :
            if ',' in col :
                colss.append(col.replace(',','-'))
            elif ']' in col :
                colss.append(col.replace(']',')'))
            else :
                colss.append(col)
        test.columns = colss
    
    
    
    return train,test


In [234]:
city_hsin_train = city_hsin[city_hsin['TRADE_YM'] <= 10700]
city_hsin_test = city_hsin[city_hsin['TRADE_YM'] > 10700]
city_kao_train = city_kao[city_kao['TRADE_YM'] <= 10700]
city_kao_test = city_kao[city_kao['TRADE_YM'] > 10700]
city_newtaipei_train = city_newtaipei[city_newtaipei['TRADE_YM'] <= 10700]
city_newtaipei_test = city_newtaipei[city_newtaipei['TRADE_YM'] > 10700]
city_taichung_train = city_taichung[city_taichung['TRADE_YM'] <= 10700]
city_taichung_test = city_taichung[city_taichung['TRADE_YM'] > 10700]
city_tainan_train = city_tainan[city_tainan['TRADE_YM'] <= 10700]
city_tainan_test = city_tainan[city_tainan['TRADE_YM'] > 10700]
city_taipei_train = city_taipei[city_taipei['TRADE_YM'] <= 10700]
city_taipei_test = city_taipei[city_taipei['TRADE_YM'] > 10700]
city_taoyuan_train = city_taoyuan[city_taoyuan['TRADE_YM'] <= 10700]
city_taoyuan_test = city_taoyuan[city_taoyuan['TRADE_YM'] > 10700]

In [235]:
city_hsin_test_ans = city_hsin_test['BUILD_U_PRICE.ADJ']
city_hsin_test = city_hsin_test.drop('BUILD_U_PRICE.ADJ',axis=1)

In [236]:
city_hsin_all = pd.concat([city_hsin_train,city_hsin_test])

In [237]:
train,test = transform(city_hsin_all)

(43203, 154)
(43203, 150)


In [238]:
xgb_model = xgb.XGBRegressor(n_estimators=600,objective = 'reg:squarederror',subsample=0.8,
                             max_depth=10,learning_rate=0.01,gamma=0.95,
                            colsample_bytree=0.99,reg_lambda=0.7,min_child_weight=0.9,
                             reg_alpha=0,n_jobs=5,verbosity=2,seed=1102)


light_model = lgb.LGBMRegressor(boosting_type='gbdt',learning_rate=0.01,num_leaves=100,max_depth=20,subsample=0.88,
                                    colsample_bytree=0.4,reg_alpha=0.1,reg_lambda=0.4,n_estimators=600,n_jobs=5)


cat_model = cat.CatBoostRegressor(n_estimators=600,one_hot_max_size=2, depth=8, l2_leaf_reg=3,
                                 random_seed=1102)


forest_model = RandomForestRegressor(n_estimators=200,n_jobs=5,max_depth=12,random_state=1102)


##### for xgboost 

X_train, X_test, y_train, y_test = train_test_split(train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1), train['BUILD_U_PRICE.ADJ'], test_size=0.15, random_state=112)    

################################## for xgboost 


print(X_train.shape)



xgb_mo = xgb_model.fit(X_train,y_train)
lig_mo = light_model.fit(X_train,y_train)
cat_mo = cat_model.fit(X_train,y_train)
#rf_mo = forest_model.fit(X_train,y_train)

#f_cv = KFold(n_splits=5)

#cv = cross_val_score(light_model,X_train,y_train,cv=f_cv,n_jobs=5)


(30528, 191)
[16:14:56] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 276 extra nodes, 0 pruned nodes, max_depth=10
[16:14:57] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 288 extra nodes, 8 pruned nodes, max_depth=10
[16:14:57] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 292 extra nodes, 6 pruned nodes, max_depth=10
[16:14:57] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 266 extra nodes, 8 pruned nodes, max_depth=10
[16:14:57] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 284 extra nodes, 12 pruned nodes, max_depth=10
[16:14:57] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 270 extra nodes, 10 prun

[16:15:05] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 500 extra nodes, 20 pruned nodes, max_depth=10
[16:15:05] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 470 extra nodes, 10 pruned nodes, max_depth=10
[16:15:06] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 462 extra nodes, 10 pruned nodes, max_depth=10
[16:15:06] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 478 extra nodes, 10 pruned nodes, max_depth=10
[16:15:06] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 472 extra nodes, 24 pruned nodes, max_depth=10
[16:15:06] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 494 extra nodes, 22 pruned nodes,

[16:15:15] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 686 extra nodes, 34 pruned nodes, max_depth=10
[16:15:15] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 658 extra nodes, 36 pruned nodes, max_depth=10
[16:15:15] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 662 extra nodes, 48 pruned nodes, max_depth=10
[16:15:16] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 692 extra nodes, 46 pruned nodes, max_depth=10
[16:15:16] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 686 extra nodes, 40 pruned nodes, max_depth=10
[16:15:16] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 662 extra nodes, 30 pruned nodes,

[16:15:25] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 816 extra nodes, 46 pruned nodes, max_depth=10
[16:15:25] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 838 extra nodes, 68 pruned nodes, max_depth=10
[16:15:25] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 874 extra nodes, 70 pruned nodes, max_depth=10
[16:15:26] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 774 extra nodes, 60 pruned nodes, max_depth=10
[16:15:26] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 876 extra nodes, 72 pruned nodes, max_depth=10
[16:15:26] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 862 extra nodes, 80 pruned nodes,

[16:15:35] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1008 extra nodes, 146 pruned nodes, max_depth=10
[16:15:35] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 934 extra nodes, 90 pruned nodes, max_depth=10
[16:15:35] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 866 extra nodes, 92 pruned nodes, max_depth=10
[16:15:35] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 940 extra nodes, 104 pruned nodes, max_depth=10
[16:15:36] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 962 extra nodes, 126 pruned nodes, max_depth=10
[16:15:36] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 880 extra nodes, 122 pruned n

...

[16:16:50] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 404 extra nodes, 134 pruned nodes, max_depth=10
[16:16:50] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 624 extra nodes, 164 pruned nodes, max_depth=10
[16:16:50] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 564 extra nodes, 94 pruned nodes, max_depth=10
[16:16:50] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 804 extra nodes, 164 pruned nodes, max_depth=10
[16:16:51] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 492 extra nodes, 152 pruned nodes, max_depth=10
[16:16:51] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 352 extra nodes, 134 pruned n

107:	learn: 2.2139526	total: 52.9s	remaining: 4m
108:	learn: 2.2099840	total: 53.3s	remaining: 4m
109:	learn: 2.2063617	total: 53.8s	remaining: 3m 59s
110:	learn: 2.2031922	total: 54.3s	remaining: 3m 59s
111:	learn: 2.1993655	total: 54.8s	remaining: 3m 58s
112:	learn: 2.1961831	total: 55.2s	remaining: 3m 58s
113:	learn: 2.1925268	total: 55.7s	remaining: 3m 57s
114:	learn: 2.1892502	total: 56.2s	remaining: 3m 57s
115:	learn: 2.1866634	total: 56.7s	remaining: 3m 56s
116:	learn: 2.1834209	total: 57.2s	remaining: 3m 56s
117:	learn: 2.1800431	total: 57.6s	remaining: 3m 55s
118:	learn: 2.1767305	total: 58.1s	remaining: 3m 54s
119:	learn: 2.1738984	total: 58.6s	remaining: 3m 54s
120:	learn: 2.1708582	total: 59.1s	remaining: 3m 53s
121:	learn: 2.1679723	total: 59.6s	remaining: 3m 53s
122:	learn: 2.1652252	total: 1m	remaining: 3m 53s
123:	learn: 2.1624382	total: 1m	remaining: 3m 52s
124:	learn: 2.1598942	total: 1m 1s	remaining: 3m 52s
125:	learn: 2.1572955	total: 1m 1s	remaining: 3m 51s
126:	le

261:	learn: 1.9750116	total: 2m 9s	remaining: 2m 47s
262:	learn: 1.9743756	total: 2m 10s	remaining: 2m 47s
263:	learn: 1.9738804	total: 2m 10s	remaining: 2m 46s
264:	learn: 1.9732815	total: 2m 11s	remaining: 2m 46s
265:	learn: 1.9727330	total: 2m 11s	remaining: 2m 45s
266:	learn: 1.9717544	total: 2m 12s	remaining: 2m 45s
267:	learn: 1.9711083	total: 2m 12s	remaining: 2m 44s
268:	learn: 1.9707110	total: 2m 13s	remaining: 2m 44s
269:	learn: 1.9704239	total: 2m 13s	remaining: 2m 43s
270:	learn: 1.9698816	total: 2m 14s	remaining: 2m 43s
271:	learn: 1.9694203	total: 2m 14s	remaining: 2m 42s
272:	learn: 1.9688036	total: 2m 15s	remaining: 2m 42s
273:	learn: 1.9675038	total: 2m 15s	remaining: 2m 41s
274:	learn: 1.9662113	total: 2m 16s	remaining: 2m 41s
275:	learn: 1.9656350	total: 2m 16s	remaining: 2m 40s
276:	learn: 1.9647497	total: 2m 17s	remaining: 2m 40s
277:	learn: 1.9643663	total: 2m 17s	remaining: 2m 39s
278:	learn: 1.9637703	total: 2m 18s	remaining: 2m 39s
279:	learn: 1.9630737	total: 

414:	learn: 1.9012689	total: 3m 25s	remaining: 1m 31s
415:	learn: 1.9000693	total: 3m 26s	remaining: 1m 31s
416:	learn: 1.8993450	total: 3m 26s	remaining: 1m 30s
417:	learn: 1.8990844	total: 3m 27s	remaining: 1m 30s
418:	learn: 1.8982370	total: 3m 27s	remaining: 1m 29s
419:	learn: 1.8980176	total: 3m 28s	remaining: 1m 29s
420:	learn: 1.8975688	total: 3m 28s	remaining: 1m 28s
421:	learn: 1.8971851	total: 3m 29s	remaining: 1m 28s
422:	learn: 1.8969720	total: 3m 29s	remaining: 1m 27s
423:	learn: 1.8967000	total: 3m 30s	remaining: 1m 27s
424:	learn: 1.8963507	total: 3m 30s	remaining: 1m 26s
425:	learn: 1.8959912	total: 3m 31s	remaining: 1m 26s
426:	learn: 1.8957979	total: 3m 31s	remaining: 1m 25s
427:	learn: 1.8951908	total: 3m 32s	remaining: 1m 25s
428:	learn: 1.8947855	total: 3m 32s	remaining: 1m 24s
429:	learn: 1.8944774	total: 3m 33s	remaining: 1m 24s
430:	learn: 1.8940047	total: 3m 33s	remaining: 1m 23s
431:	learn: 1.8937206	total: 3m 34s	remaining: 1m 23s
432:	learn: 1.8933419	total:

569:	learn: 1.8455517	total: 4m 43s	remaining: 14.9s
570:	learn: 1.8452981	total: 4m 43s	remaining: 14.4s
571:	learn: 1.8450701	total: 4m 44s	remaining: 13.9s
572:	learn: 1.8448473	total: 4m 44s	remaining: 13.4s
573:	learn: 1.8445707	total: 4m 45s	remaining: 12.9s
574:	learn: 1.8443123	total: 4m 45s	remaining: 12.4s
575:	learn: 1.8439978	total: 4m 46s	remaining: 11.9s
576:	learn: 1.8435438	total: 4m 46s	remaining: 11.4s
577:	learn: 1.8431585	total: 4m 47s	remaining: 10.9s
578:	learn: 1.8423522	total: 4m 47s	remaining: 10.4s
579:	learn: 1.8420053	total: 4m 48s	remaining: 9.94s
580:	learn: 1.8417693	total: 4m 48s	remaining: 9.45s
581:	learn: 1.8414599	total: 4m 49s	remaining: 8.95s
582:	learn: 1.8413555	total: 4m 49s	remaining: 8.45s
583:	learn: 1.8410869	total: 4m 50s	remaining: 7.96s
584:	learn: 1.8408530	total: 4m 50s	remaining: 7.46s
585:	learn: 1.8405719	total: 4m 51s	remaining: 6.96s
586:	learn: 1.8402011	total: 4m 51s	remaining: 6.46s
587:	learn: 1.8400099	total: 4m 52s	remaining:

In [239]:
pre_li = lig_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
pre_xg = xgb_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
pre_ca = cat_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
print(
    (((abs(city_hsin_test_ans - pre_li ) / city_hsin_test_ans ) < 0.1 ).sum() ) / len(city_hsin_test_ans),
    (((abs(city_hsin_test_ans - pre_xg ) / city_hsin_test_ans ) < 0.1 ).sum() ) / len(city_hsin_test_ans),
    (((abs(city_hsin_test_ans - pre_ca ) / city_hsin_test_ans ) < 0.1 ).sum() ) / len(city_hsin_test_ans)
     )

0.6875257307533965 0.663510360916701 0.6456703718951557


In [203]:
(((abs(city_hsin_test_ans - pre ) / city_hsin_test_ans ) < 0.1 ).sum() ) / len(city_hsin_test_ans)

0.6858789625360231

In [240]:
city_kao_test_ans = city_kao_test['BUILD_U_PRICE.ADJ']
city_kao_test = city_kao_test.drop('BUILD_U_PRICE.ADJ',axis=1)
city_kao_all = pd.concat([city_kao_train,city_kao_test])
train,test = transform(city_kao_all)

xgb_model = xgb.XGBRegressor(n_estimators=600,objective = 'reg:squarederror',subsample=0.8,
                             max_depth=10,learning_rate=0.01,gamma=0.95,
                            colsample_bytree=0.99,reg_lambda=0.7,min_child_weight=0.9,
                             reg_alpha=0,n_jobs=5,verbosity=2,seed=1102)


light_model = lgb.LGBMRegressor(boosting_type='gbdt',learning_rate=0.01,num_leaves=100,max_depth=15,subsample=0.88,
                                    colsample_bytree=0.4,reg_alpha=0.1,reg_lambda=0.4,n_estimators=600,n_jobs=5)


cat_model = cat.CatBoostRegressor(n_estimators=600,one_hot_max_size=2, depth=8, l2_leaf_reg=3,
                                 random_seed=1102)


forest_model = RandomForestRegressor(n_estimators=200,n_jobs=5,max_depth=12,random_state=1102)


##### for xgboost 

X_train, X_test, y_train, y_test = train_test_split(train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1), train['BUILD_U_PRICE.ADJ'], test_size=0.2, random_state=1102)    

################################## for xgboost 


print(X_train.shape)



xgb_mo = xgb_model.fit(X_train,y_train)
lig_mo = light_model.fit(X_train,y_train)
cat_mo = cat_model.fit(X_train,y_train)
#rf_mo = forest_model.fit(X_train,y_train)

#f_cv = KFold(n_splits=5)

#cv = cross_val_score(light_model,X_train,y_train,cv=f_cv,n_jobs=5)


(87219, 154)
(87219, 154)
(57860, 203)
[16:23:35] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 498 extra nodes, 16 pruned nodes, max_depth=10
[16:23:35] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 544 extra nodes, 16 pruned nodes, max_depth=10
[16:23:36] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 518 extra nodes, 18 pruned nodes, max_depth=10
[16:23:36] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 534 extra nodes, 10 pruned nodes, max_depth=10
[16:23:37] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 530 extra nodes, 18 pruned nodes, max_depth=10
[16:23:37] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 r

[16:23:54] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 726 extra nodes, 40 pruned nodes, max_depth=10
[16:23:55] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 726 extra nodes, 36 pruned nodes, max_depth=10
[16:23:55] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 738 extra nodes, 42 pruned nodes, max_depth=10
[16:23:55] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 700 extra nodes, 42 pruned nodes, max_depth=10
[16:23:56] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 790 extra nodes, 44 pruned nodes, max_depth=10
[16:23:56] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 712 extra nodes, 38 pruned nodes,

[16:24:14] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1026 extra nodes, 94 pruned nodes, max_depth=10
[16:24:15] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 960 extra nodes, 38 pruned nodes, max_depth=10
[16:24:15] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 976 extra nodes, 72 pruned nodes, max_depth=10
[16:24:16] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 946 extra nodes, 44 pruned nodes, max_depth=10
[16:24:16] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 966 extra nodes, 56 pruned nodes, max_depth=10
[16:24:17] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 972 extra nodes, 66 pruned nodes

[16:24:35] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1184 extra nodes, 78 pruned nodes, max_depth=10
[16:24:35] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1072 extra nodes, 54 pruned nodes, max_depth=10
[16:24:36] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1082 extra nodes, 62 pruned nodes, max_depth=10
[16:24:36] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1044 extra nodes, 60 pruned nodes, max_depth=10
[16:24:37] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1144 extra nodes, 74 pruned nodes, max_depth=10
[16:24:37] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1130 extra nodes, 80 pruned 

[16:24:55] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1130 extra nodes, 92 pruned nodes, max_depth=10
[16:24:56] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1102 extra nodes, 126 pruned nodes, max_depth=10
[16:24:56] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1302 extra nodes, 124 pruned nodes, max_depth=10
[16:24:56] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1138 extra nodes, 68 pruned nodes, max_depth=10
[16:24:57] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1206 extra nodes, 94 pruned nodes, max_depth=10
[16:24:57] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1104 extra nodes, 98 prune

...

[16:27:34] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 820 extra nodes, 126 pruned nodes, max_depth=10
[16:27:34] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 664 extra nodes, 136 pruned nodes, max_depth=10
[16:27:35] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 608 extra nodes, 96 pruned nodes, max_depth=10
[16:27:35] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 722 extra nodes, 110 pruned nodes, max_depth=10
[16:27:36] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 768 extra nodes, 112 pruned nodes, max_depth=10
[16:27:36] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 920 extra nodes, 160 pruned n

114:	learn: 1.9300055	total: 15.1s	remaining: 1m 3s
115:	learn: 1.9280536	total: 15.2s	remaining: 1m 3s
116:	learn: 1.9261869	total: 15.4s	remaining: 1m 3s
117:	learn: 1.9241367	total: 15.5s	remaining: 1m 3s
118:	learn: 1.9219216	total: 15.6s	remaining: 1m 3s
119:	learn: 1.9196924	total: 15.7s	remaining: 1m 2s
120:	learn: 1.9181202	total: 15.9s	remaining: 1m 2s
121:	learn: 1.9165672	total: 16s	remaining: 1m 2s
122:	learn: 1.9145893	total: 16.1s	remaining: 1m 2s
123:	learn: 1.9119909	total: 16.2s	remaining: 1m 2s
124:	learn: 1.9105683	total: 16.4s	remaining: 1m 2s
125:	learn: 1.9091016	total: 16.5s	remaining: 1m 2s
126:	learn: 1.9074095	total: 16.6s	remaining: 1m 1s
127:	learn: 1.9057112	total: 16.7s	remaining: 1m 1s
128:	learn: 1.9045011	total: 16.9s	remaining: 1m 1s
129:	learn: 1.9030887	total: 17s	remaining: 1m 1s
130:	learn: 1.9013172	total: 17.1s	remaining: 1m 1s
131:	learn: 1.9000220	total: 17.2s	remaining: 1m 1s
132:	learn: 1.8985056	total: 17.3s	remaining: 1m
133:	learn: 1.89712

274:	learn: 1.7968018	total: 34.9s	remaining: 41.3s
275:	learn: 1.7962808	total: 35.1s	remaining: 41.2s
276:	learn: 1.7959011	total: 35.2s	remaining: 41s
277:	learn: 1.7953410	total: 35.3s	remaining: 40.9s
278:	learn: 1.7949474	total: 35.4s	remaining: 40.8s
279:	learn: 1.7945456	total: 35.6s	remaining: 40.6s
280:	learn: 1.7940798	total: 35.7s	remaining: 40.5s
281:	learn: 1.7931401	total: 35.8s	remaining: 40.4s
282:	learn: 1.7927584	total: 35.9s	remaining: 40.3s
283:	learn: 1.7924798	total: 36.1s	remaining: 40.1s
284:	learn: 1.7920386	total: 36.2s	remaining: 40s
285:	learn: 1.7916280	total: 36.3s	remaining: 39.9s
286:	learn: 1.7910266	total: 36.4s	remaining: 39.7s
287:	learn: 1.7904734	total: 36.5s	remaining: 39.6s
288:	learn: 1.7900848	total: 36.7s	remaining: 39.5s
289:	learn: 1.7896279	total: 36.8s	remaining: 39.3s
290:	learn: 1.7890795	total: 36.9s	remaining: 39.2s
291:	learn: 1.7885312	total: 37s	remaining: 39.1s
292:	learn: 1.7882419	total: 37.2s	remaining: 38.9s
293:	learn: 1.7878

434:	learn: 1.7312185	total: 55.1s	remaining: 20.9s
435:	learn: 1.7311485	total: 55.2s	remaining: 20.8s
436:	learn: 1.7308002	total: 55.4s	remaining: 20.7s
437:	learn: 1.7298814	total: 55.5s	remaining: 20.5s
438:	learn: 1.7293107	total: 55.6s	remaining: 20.4s
439:	learn: 1.7290049	total: 55.7s	remaining: 20.3s
440:	learn: 1.7286976	total: 55.9s	remaining: 20.1s
441:	learn: 1.7283709	total: 56s	remaining: 20s
442:	learn: 1.7281497	total: 56.1s	remaining: 19.9s
443:	learn: 1.7279905	total: 56.2s	remaining: 19.8s
444:	learn: 1.7277508	total: 56.3s	remaining: 19.6s
445:	learn: 1.7274344	total: 56.5s	remaining: 19.5s
446:	learn: 1.7268131	total: 56.6s	remaining: 19.4s
447:	learn: 1.7266373	total: 56.7s	remaining: 19.2s
448:	learn: 1.7263137	total: 56.8s	remaining: 19.1s
449:	learn: 1.7258695	total: 57s	remaining: 19s
450:	learn: 1.7256175	total: 57.1s	remaining: 18.9s
451:	learn: 1.7254288	total: 57.2s	remaining: 18.7s
452:	learn: 1.7250956	total: 57.4s	remaining: 18.6s
453:	learn: 1.724837

592:	learn: 1.6802174	total: 1m 16s	remaining: 900ms
593:	learn: 1.6798013	total: 1m 16s	remaining: 772ms
594:	learn: 1.6792256	total: 1m 16s	remaining: 643ms
595:	learn: 1.6790594	total: 1m 16s	remaining: 514ms
596:	learn: 1.6787240	total: 1m 16s	remaining: 386ms
597:	learn: 1.6785292	total: 1m 16s	remaining: 257ms
598:	learn: 1.6782290	total: 1m 17s	remaining: 129ms
599:	learn: 1.6779313	total: 1m 17s	remaining: 0us


In [242]:
pre_li = lig_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
pre_xg = xgb_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
pre_ca = cat_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
print(
    (((abs(city_kao_test_ans - pre_li ) / city_kao_test_ans ) < 0.1 ).sum() ) / len(city_kao_test_ans),
    (((abs(city_kao_test_ans - pre_xg ) / city_kao_test_ans ) < 0.1 ).sum() ) / len(city_kao_test_ans),
    (((abs(city_kao_test_ans - pre_ca ) / city_kao_test_ans ) < 0.1 ).sum() ) / len(city_kao_test_ans)
     )

0.5198036446775604 0.5086409790868133 0.49989913253984264


In [213]:
pre = lig_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
print('hit_rate',
    (((abs(city_kao_test_ans - pre ) / city_kao_test_ans ) < 0.1 ).sum() ) / len(city_kao_test_ans))

hit_rate 0.5198036446775604


In [243]:
city_newtaipei_test_ans = city_newtaipei_test['BUILD_U_PRICE.ADJ']
city_newtaipei_test = city_newtaipei_test.drop('BUILD_U_PRICE.ADJ',axis=1)
city_newtaipei_all = pd.concat([city_newtaipei_train,city_newtaipei_test])
train,test = transform(city_newtaipei_all)

xgb_model = xgb.XGBRegressor(n_estimators=600,objective = 'reg:squarederror',subsample=0.8,
                             max_depth=10,learning_rate=0.01,gamma=0.95,
                            colsample_bytree=0.99,reg_lambda=0.7,min_child_weight=0.9,
                             reg_alpha=0,n_jobs=5,verbosity=2,seed=1102)


light_model = lgb.LGBMRegressor(boosting_type='gbdt',learning_rate=0.01,num_leaves=100,max_depth=15,subsample=0.88,
                                    colsample_bytree=0.4,reg_alpha=0.1,reg_lambda=0.4,n_estimators=600,n_jobs=5)


cat_model = cat.CatBoostRegressor(n_estimators=600,one_hot_max_size=2, depth=8, l2_leaf_reg=3,
                                 random_seed=1102)


forest_model = RandomForestRegressor(n_estimators=200,n_jobs=5,max_depth=12,random_state=1102)


##### for xgboost 

X_train, X_test, y_train, y_test = train_test_split(train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1), train['BUILD_U_PRICE.ADJ'], test_size=0.2, random_state=1102)    

################################## for xgboost 


print(X_train.shape)



xgb_mo = xgb_model.fit(X_train,y_train)
lig_mo = light_model.fit(X_train,y_train)
cat_mo = cat_model.fit(X_train,y_train)
#rf_mo = forest_model.fit(X_train,y_train)

#f_cv = KFold(n_splits=5)

#cv = cross_val_score(light_model,X_train,y_train,cv=f_cv,n_jobs=5)

(158143, 154)
(158143, 154)
(104290, 196)
[16:30:12] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 536 extra nodes, 2 pruned nodes, max_depth=10
[16:30:13] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 512 extra nodes, 6 pruned nodes, max_depth=10
[16:30:13] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 518 extra nodes, 4 pruned nodes, max_depth=10
[16:30:14] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 524 extra nodes, 8 pruned nodes, max_depth=10
[16:30:15] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 526 extra nodes, 12 pruned nodes, max_depth=10
[16:30:15] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 ro

[16:30:45] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 778 extra nodes, 20 pruned nodes, max_depth=10
[16:30:45] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 772 extra nodes, 8 pruned nodes, max_depth=10
[16:30:46] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 776 extra nodes, 14 pruned nodes, max_depth=10
[16:30:47] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 816 extra nodes, 12 pruned nodes, max_depth=10
[16:30:47] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 776 extra nodes, 8 pruned nodes, max_depth=10
[16:30:48] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 764 extra nodes, 10 pruned nodes, m

[16:31:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1052 extra nodes, 18 pruned nodes, max_depth=10
[16:31:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1038 extra nodes, 8 pruned nodes, max_depth=10
[16:31:20] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1098 extra nodes, 32 pruned nodes, max_depth=10
[16:31:20] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1084 extra nodes, 38 pruned nodes, max_depth=10
[16:31:21] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 998 extra nodes, 18 pruned nodes, max_depth=10
[16:31:22] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1122 extra nodes, 6 pruned nod

[16:31:52] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1186 extra nodes, 34 pruned nodes, max_depth=10
[16:31:53] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1320 extra nodes, 34 pruned nodes, max_depth=10
[16:31:54] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1286 extra nodes, 26 pruned nodes, max_depth=10
[16:31:54] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1160 extra nodes, 24 pruned nodes, max_depth=10
[16:31:55] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1156 extra nodes, 34 pruned nodes, max_depth=10
[16:31:56] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1264 extra nodes, 32 pruned 

[16:32:26] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1240 extra nodes, 34 pruned nodes, max_depth=10
[16:32:27] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1308 extra nodes, 34 pruned nodes, max_depth=10
[16:32:28] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1462 extra nodes, 40 pruned nodes, max_depth=10
[16:32:29] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1382 extra nodes, 34 pruned nodes, max_depth=10
[16:32:29] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1368 extra nodes, 40 pruned nodes, max_depth=10
[16:32:30] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1322 extra nodes, 42 pruned 

...

[16:37:05] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 752 extra nodes, 56 pruned nodes, max_depth=10
[16:37:06] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 816 extra nodes, 58 pruned nodes, max_depth=10
[16:37:07] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1122 extra nodes, 78 pruned nodes, max_depth=10
[16:37:08] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 850 extra nodes, 60 pruned nodes, max_depth=10
[16:37:08] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1250 extra nodes, 92 pruned nodes, max_depth=10
[16:37:09] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 978 extra nodes, 70 pruned node

120:	learn: 3.8242909	total: 20.3s	remaining: 1m 20s
121:	learn: 3.8201470	total: 20.4s	remaining: 1m 20s
122:	learn: 3.8151873	total: 20.6s	remaining: 1m 19s
123:	learn: 3.8114234	total: 20.8s	remaining: 1m 19s
124:	learn: 3.8064842	total: 20.9s	remaining: 1m 19s
125:	learn: 3.8030159	total: 21.1s	remaining: 1m 19s
126:	learn: 3.7989713	total: 21.3s	remaining: 1m 19s
127:	learn: 3.7945507	total: 21.4s	remaining: 1m 19s
128:	learn: 3.7906186	total: 21.6s	remaining: 1m 18s
129:	learn: 3.7871391	total: 21.8s	remaining: 1m 18s
130:	learn: 3.7837816	total: 21.9s	remaining: 1m 18s
131:	learn: 3.7800080	total: 22.1s	remaining: 1m 18s
132:	learn: 3.7763266	total: 22.2s	remaining: 1m 18s
133:	learn: 3.7735323	total: 22.4s	remaining: 1m 17s
134:	learn: 3.7699354	total: 22.6s	remaining: 1m 17s
135:	learn: 3.7667304	total: 22.7s	remaining: 1m 17s
136:	learn: 3.7632802	total: 22.9s	remaining: 1m 17s
137:	learn: 3.7604801	total: 23s	remaining: 1m 17s
138:	learn: 3.7562025	total: 23.2s	remaining: 1m

279:	learn: 3.4969765	total: 45.6s	remaining: 52.1s
280:	learn: 3.4962731	total: 45.8s	remaining: 51.9s
281:	learn: 3.4951667	total: 45.9s	remaining: 51.8s
282:	learn: 3.4936617	total: 46.1s	remaining: 51.6s
283:	learn: 3.4926898	total: 46.2s	remaining: 51.5s
284:	learn: 3.4915886	total: 46.4s	remaining: 51.3s
285:	learn: 3.4904418	total: 46.6s	remaining: 51.1s
286:	learn: 3.4890252	total: 46.7s	remaining: 50.9s
287:	learn: 3.4880774	total: 46.9s	remaining: 50.8s
288:	learn: 3.4861014	total: 47s	remaining: 50.6s
289:	learn: 3.4847737	total: 47.2s	remaining: 50.4s
290:	learn: 3.4831807	total: 47.3s	remaining: 50.3s
291:	learn: 3.4824124	total: 47.5s	remaining: 50.1s
292:	learn: 3.4817057	total: 47.7s	remaining: 49.9s
293:	learn: 3.4805104	total: 47.8s	remaining: 49.8s
294:	learn: 3.4792947	total: 48s	remaining: 49.6s
295:	learn: 3.4779341	total: 48.1s	remaining: 49.4s
296:	learn: 3.4767263	total: 48.3s	remaining: 49.3s
297:	learn: 3.4753417	total: 48.5s	remaining: 49.1s
298:	learn: 3.47

438:	learn: 3.3427012	total: 1m 10s	remaining: 26s
439:	learn: 3.3418948	total: 1m 11s	remaining: 25.8s
440:	learn: 3.3415593	total: 1m 11s	remaining: 25.7s
441:	learn: 3.3409437	total: 1m 11s	remaining: 25.5s
442:	learn: 3.3398632	total: 1m 11s	remaining: 25.4s
443:	learn: 3.3389056	total: 1m 11s	remaining: 25.2s
444:	learn: 3.3383590	total: 1m 11s	remaining: 25s
445:	learn: 3.3375386	total: 1m 12s	remaining: 24.9s
446:	learn: 3.3364226	total: 1m 12s	remaining: 24.7s
447:	learn: 3.3352746	total: 1m 12s	remaining: 24.6s
448:	learn: 3.3343170	total: 1m 12s	remaining: 24.4s
449:	learn: 3.3334012	total: 1m 12s	remaining: 24.3s
450:	learn: 3.3326552	total: 1m 12s	remaining: 24.1s
451:	learn: 3.3319147	total: 1m 13s	remaining: 23.9s
452:	learn: 3.3312740	total: 1m 13s	remaining: 23.8s
453:	learn: 3.3306013	total: 1m 13s	remaining: 23.6s
454:	learn: 3.3297856	total: 1m 13s	remaining: 23.4s
455:	learn: 3.3291867	total: 1m 13s	remaining: 23.3s
456:	learn: 3.3282805	total: 1m 13s	remaining: 23.

595:	learn: 3.2379739	total: 1m 35s	remaining: 643ms
596:	learn: 3.2375576	total: 1m 35s	remaining: 482ms
597:	learn: 3.2371788	total: 1m 36s	remaining: 322ms
598:	learn: 3.2367543	total: 1m 36s	remaining: 161ms
599:	learn: 3.2363234	total: 1m 36s	remaining: 0us


In [244]:
pre_li = lig_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
pre_xg = xgb_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
pre_ca = cat_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
print(
    (((abs(city_newtaipei_test_ans - pre_li ) / city_newtaipei_test_ans ) < 0.1 ).sum() ) / len(city_newtaipei_test_ans),
    (((abs(city_newtaipei_test_ans - pre_xg ) / city_newtaipei_test_ans ) < 0.1 ).sum() ) / len(city_newtaipei_test_ans),
    (((abs(city_newtaipei_test_ans - pre_ca ) / city_newtaipei_test_ans ) < 0.1 ).sum() ) / len(city_newtaipei_test_ans)
     )

0.6520266618627274 0.6333993874977482 0.613979463159791


In [216]:
pre = lig_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
print('hit_rate',
    (((abs(city_newtaipei_test_ans - pre ) / city_newtaipei_test_ans ) < 0.1 ).sum() ) / len(city_newtaipei_test_ans))

hit_rate 0.6520266618627274


In [245]:
city_taichung_test_ans = city_taichung_test['BUILD_U_PRICE.ADJ']
city_taichung_test = city_taichung_test.drop('BUILD_U_PRICE.ADJ',axis=1)
city_taichung_all = pd.concat([city_taichung_train,city_taichung_test])
train,test = transform(city_taichung_all)

xgb_model = xgb.XGBRegressor(n_estimators=600,objective = 'reg:squarederror',subsample=0.8,
                             max_depth=10,learning_rate=0.01,gamma=0.95,
                            colsample_bytree=0.99,reg_lambda=0.7,min_child_weight=0.9,
                             reg_alpha=0,n_jobs=5,verbosity=2,seed=1102)


light_model = lgb.LGBMRegressor(boosting_type='gbdt',learning_rate=0.01,num_leaves=100,max_depth=15,subsample=0.88,
                                    colsample_bytree=0.4,reg_alpha=0.1,reg_lambda=0.4,n_estimators=600,n_jobs=5)


cat_model = cat.CatBoostRegressor(n_estimators=600,one_hot_max_size=2, depth=8, l2_leaf_reg=3,
                                 random_seed=1102)


forest_model = RandomForestRegressor(n_estimators=200,n_jobs=5,max_depth=12,random_state=1102)


##### for xgboost 

X_train, X_test, y_train, y_test = train_test_split(train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1), train['BUILD_U_PRICE.ADJ'], test_size=0.2, random_state=1102)    

################################## for xgboost 


print(X_train.shape)



xgb_mo = xgb_model.fit(X_train,y_train)
lig_mo = light_model.fit(X_train,y_train)
cat_mo = cat_model.fit(X_train,y_train)
#rf_mo = forest_model.fit(X_train,y_train)

#f_cv = KFold(n_splits=5)

#cv = cross_val_score(light_model,X_train,y_train,cv=f_cv,n_jobs=5)

(87772, 154)
(87772, 154)
(57908, 201)
[16:39:35] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 372 extra nodes, 12 pruned nodes, max_depth=10
[16:39:36] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 370 extra nodes, 2 pruned nodes, max_depth=10
[16:39:36] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 376 extra nodes, 18 pruned nodes, max_depth=10
[16:39:36] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 372 extra nodes, 10 pruned nodes, max_depth=10
[16:39:37] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 386 extra nodes, 8 pruned nodes, max_depth=10
[16:39:37] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roo

[16:39:53] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 602 extra nodes, 12 pruned nodes, max_depth=10
[16:39:54] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 646 extra nodes, 6 pruned nodes, max_depth=10
[16:39:54] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 630 extra nodes, 18 pruned nodes, max_depth=10
[16:39:54] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 608 extra nodes, 24 pruned nodes, max_depth=10
[16:39:55] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 638 extra nodes, 12 pruned nodes, max_depth=10
[16:39:55] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 664 extra nodes, 16 pruned nodes, 

[16:40:12] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 916 extra nodes, 34 pruned nodes, max_depth=10
[16:40:12] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 876 extra nodes, 46 pruned nodes, max_depth=10
[16:40:13] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 838 extra nodes, 32 pruned nodes, max_depth=10
[16:40:13] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 892 extra nodes, 24 pruned nodes, max_depth=10
[16:40:14] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 946 extra nodes, 38 pruned nodes, max_depth=10
[16:40:14] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 934 extra nodes, 32 pruned nodes,

[16:40:31] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1050 extra nodes, 44 pruned nodes, max_depth=10
[16:40:32] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1056 extra nodes, 60 pruned nodes, max_depth=10
[16:40:32] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1154 extra nodes, 44 pruned nodes, max_depth=10
[16:40:32] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1100 extra nodes, 62 pruned nodes, max_depth=10
[16:40:33] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1114 extra nodes, 56 pruned nodes, max_depth=10
[16:40:33] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1146 extra nodes, 46 pruned 

[16:40:50] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1164 extra nodes, 78 pruned nodes, max_depth=10
[16:40:51] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1092 extra nodes, 62 pruned nodes, max_depth=10
[16:40:51] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1126 extra nodes, 66 pruned nodes, max_depth=10
[16:40:51] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1020 extra nodes, 62 pruned nodes, max_depth=10
[16:40:52] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1142 extra nodes, 76 pruned nodes, max_depth=10
[16:40:52] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1162 extra nodes, 56 pruned 

...

[16:43:24] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 680 extra nodes, 156 pruned nodes, max_depth=10
[16:43:24] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 818 extra nodes, 146 pruned nodes, max_depth=10
[16:43:25] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 656 extra nodes, 92 pruned nodes, max_depth=10
[16:43:25] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 720 extra nodes, 108 pruned nodes, max_depth=10
[16:43:25] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 816 extra nodes, 176 pruned nodes, max_depth=10
[16:43:26] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 832 extra nodes, 120 pruned n

121:	learn: 2.1926377	total: 16.6s	remaining: 1m 5s
122:	learn: 2.1910273	total: 16.8s	remaining: 1m 4s
123:	learn: 2.1893387	total: 16.9s	remaining: 1m 4s
124:	learn: 2.1875665	total: 17s	remaining: 1m 4s
125:	learn: 2.1857115	total: 17.2s	remaining: 1m 4s
126:	learn: 2.1842490	total: 17.3s	remaining: 1m 4s
127:	learn: 2.1827685	total: 17.4s	remaining: 1m 4s
128:	learn: 2.1810775	total: 17.5s	remaining: 1m 4s
129:	learn: 2.1794242	total: 17.7s	remaining: 1m 3s
130:	learn: 2.1781739	total: 17.8s	remaining: 1m 3s
131:	learn: 2.1763096	total: 17.9s	remaining: 1m 3s
132:	learn: 2.1745610	total: 18.1s	remaining: 1m 3s
133:	learn: 2.1729437	total: 18.2s	remaining: 1m 3s
134:	learn: 2.1710400	total: 18.3s	remaining: 1m 3s
135:	learn: 2.1693294	total: 18.5s	remaining: 1m 3s
136:	learn: 2.1671369	total: 18.6s	remaining: 1m 2s
137:	learn: 2.1654218	total: 18.7s	remaining: 1m 2s
138:	learn: 2.1639524	total: 18.9s	remaining: 1m 2s
139:	learn: 2.1627428	total: 19s	remaining: 1m 2s
140:	learn: 2.16

281:	learn: 2.0410371	total: 36.9s	remaining: 41.7s
282:	learn: 2.0406508	total: 37.1s	remaining: 41.5s
283:	learn: 2.0401009	total: 37.2s	remaining: 41.4s
284:	learn: 2.0395645	total: 37.3s	remaining: 41.2s
285:	learn: 2.0388960	total: 37.4s	remaining: 41.1s
286:	learn: 2.0383678	total: 37.6s	remaining: 41s
287:	learn: 2.0379697	total: 37.7s	remaining: 40.8s
288:	learn: 2.0375096	total: 37.8s	remaining: 40.7s
289:	learn: 2.0367289	total: 38s	remaining: 40.6s
290:	learn: 2.0363404	total: 38.1s	remaining: 40.4s
291:	learn: 2.0360685	total: 38.2s	remaining: 40.3s
292:	learn: 2.0355343	total: 38.4s	remaining: 40.2s
293:	learn: 2.0351813	total: 38.5s	remaining: 40.1s
294:	learn: 2.0339944	total: 38.6s	remaining: 39.9s
295:	learn: 2.0337114	total: 38.8s	remaining: 39.8s
296:	learn: 2.0331110	total: 38.9s	remaining: 39.7s
297:	learn: 2.0325497	total: 39s	remaining: 39.5s
298:	learn: 2.0321660	total: 39.1s	remaining: 39.4s
299:	learn: 2.0316724	total: 39.3s	remaining: 39.3s
300:	learn: 2.0309

441:	learn: 1.9624526	total: 56.6s	remaining: 20.2s
442:	learn: 1.9620621	total: 56.7s	remaining: 20.1s
443:	learn: 1.9616315	total: 56.9s	remaining: 20s
444:	learn: 1.9612069	total: 57s	remaining: 19.8s
445:	learn: 1.9608202	total: 57.1s	remaining: 19.7s
446:	learn: 1.9602373	total: 57.2s	remaining: 19.6s
447:	learn: 1.9598826	total: 57.3s	remaining: 19.5s
448:	learn: 1.9593787	total: 57.5s	remaining: 19.3s
449:	learn: 1.9590862	total: 57.6s	remaining: 19.2s
450:	learn: 1.9586556	total: 57.7s	remaining: 19.1s
451:	learn: 1.9582912	total: 57.8s	remaining: 18.9s
452:	learn: 1.9578700	total: 58s	remaining: 18.8s
453:	learn: 1.9576559	total: 58.1s	remaining: 18.7s
454:	learn: 1.9573132	total: 58.2s	remaining: 18.6s
455:	learn: 1.9569689	total: 58.3s	remaining: 18.4s
456:	learn: 1.9564249	total: 58.5s	remaining: 18.3s
457:	learn: 1.9560772	total: 58.6s	remaining: 18.2s
458:	learn: 1.9556365	total: 58.7s	remaining: 18s
459:	learn: 1.9548798	total: 58.9s	remaining: 17.9s
460:	learn: 1.954636

599:	learn: 1.9060612	total: 1m 16s	remaining: 0us


In [246]:
pre_li = lig_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
pre_xg = xgb_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
pre_ca = cat_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
print(
    (((abs(city_taichung_test_ans - pre_li ) / city_taichung_test_ans ) < 0.1 ).sum() ) / len(city_taichung_test_ans),
    (((abs(city_taichung_test_ans - pre_xg ) / city_taichung_test_ans ) < 0.1 ).sum() ) / len(city_taichung_test_ans),
    (((abs(city_taichung_test_ans - pre_ca ) / city_taichung_test_ans ) < 0.1 ).sum() ) / len(city_taichung_test_ans)
     )

0.594761471467568 0.5999610035096842 0.5862472377486027


In [218]:
pre = lig_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
print('hit_rate',
    (((abs(city_taichung_test_ans - pre ) / city_taichung_test_ans ) < 0.1 ).sum() ) / len(city_taichung_test_ans))

hit_rate 0.594761471467568


In [247]:
city_tainan_test_ans = city_tainan_test['BUILD_U_PRICE.ADJ']
city_tainan_test = city_tainan_test.drop('BUILD_U_PRICE.ADJ',axis=1)
city_tainan_all = pd.concat([city_tainan_train,city_tainan_test])
train,test = transform(city_tainan_all)

xgb_model = xgb.XGBRegressor(n_estimators=600,objective = 'reg:squarederror',subsample=0.8,
                             max_depth=10,learning_rate=0.01,gamma=0.95,
                            colsample_bytree=0.99,reg_lambda=0.7,min_child_weight=0.9,
                             reg_alpha=0,n_jobs=5,verbosity=2,seed=1102)


light_model = lgb.LGBMRegressor(boosting_type='gbdt',learning_rate=0.01,num_leaves=100,max_depth=15,subsample=0.88,
                                    colsample_bytree=0.4,reg_alpha=0.1,reg_lambda=0.4,n_estimators=600,n_jobs=5)


cat_model = cat.CatBoostRegressor(n_estimators=600,one_hot_max_size=2, depth=8, l2_leaf_reg=3,
                                 random_seed=1102)


forest_model = RandomForestRegressor(n_estimators=200,n_jobs=5,max_depth=12,random_state=1102)


##### for xgboost 

X_train, X_test, y_train, y_test = train_test_split(train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1), train['BUILD_U_PRICE.ADJ'], test_size=0.2, random_state=1102)    

################################## for xgboost 


print(X_train.shape)



xgb_mo = xgb_model.fit(X_train,y_train)
lig_mo = light_model.fit(X_train,y_train)
cat_mo = cat_model.fit(X_train,y_train)
#rf_mo = forest_model.fit(X_train,y_train)

#f_cv = KFold(n_splits=5)

#cv = cross_val_score(light_model,X_train,y_train,cv=f_cv,n_jobs=5)

(32581, 154)
(32581, 150)
(20952, 191)
[16:45:14] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 324 extra nodes, 12 pruned nodes, max_depth=10
[16:45:14] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 324 extra nodes, 12 pruned nodes, max_depth=10
[16:45:14] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 338 extra nodes, 10 pruned nodes, max_depth=10
[16:45:15] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 308 extra nodes, 8 pruned nodes, max_depth=10
[16:45:15] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 334 extra nodes, 14 pruned nodes, max_depth=10
[16:45:15] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 ro

[16:45:21] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 494 extra nodes, 16 pruned nodes, max_depth=10
[16:45:21] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 476 extra nodes, 40 pruned nodes, max_depth=10
[16:45:21] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 518 extra nodes, 46 pruned nodes, max_depth=10
[16:45:21] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 494 extra nodes, 26 pruned nodes, max_depth=10
[16:45:21] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 484 extra nodes, 32 pruned nodes, max_depth=10
[16:45:21] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 532 extra nodes, 42 pruned nodes,

[16:45:27] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 772 extra nodes, 72 pruned nodes, max_depth=10
[16:45:27] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 668 extra nodes, 80 pruned nodes, max_depth=10
[16:45:28] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 704 extra nodes, 84 pruned nodes, max_depth=10
[16:45:28] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 666 extra nodes, 84 pruned nodes, max_depth=10
[16:45:28] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 668 extra nodes, 82 pruned nodes, max_depth=10
[16:45:28] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 674 extra nodes, 84 pruned nodes,

[16:45:34] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 958 extra nodes, 132 pruned nodes, max_depth=10
[16:45:34] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 662 extra nodes, 72 pruned nodes, max_depth=10
[16:45:34] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 758 extra nodes, 122 pruned nodes, max_depth=10
[16:45:34] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 756 extra nodes, 86 pruned nodes, max_depth=10
[16:45:34] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 772 extra nodes, 98 pruned nodes, max_depth=10
[16:45:35] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 868 extra nodes, 106 pruned nod

[16:45:40] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 726 extra nodes, 134 pruned nodes, max_depth=10
[16:45:41] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 744 extra nodes, 128 pruned nodes, max_depth=10
[16:45:41] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 764 extra nodes, 130 pruned nodes, max_depth=10
[16:45:41] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 838 extra nodes, 164 pruned nodes, max_depth=10
[16:45:41] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 862 extra nodes, 144 pruned nodes, max_depth=10
[16:45:41] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 766 extra nodes, 120 pruned 

...

[16:46:26] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 392 extra nodes, 86 pruned nodes, max_depth=10
[16:46:26] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 522 extra nodes, 164 pruned nodes, max_depth=10
[16:46:26] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 426 extra nodes, 90 pruned nodes, max_depth=10
[16:46:26] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 706 extra nodes, 160 pruned nodes, max_depth=10
[16:46:26] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 400 extra nodes, 158 pruned nodes, max_depth=10
[16:46:26] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 486 extra nodes, 100 pruned no

0:	learn: 4.5385383	total: 421ms	remaining: 4m 12s
1:	learn: 4.4412895	total: 858ms	remaining: 4m 16s
2:	learn: 4.3437952	total: 1.3s	remaining: 4m 18s
3:	learn: 4.2561197	total: 1.73s	remaining: 4m 18s
4:	learn: 4.1679319	total: 2.17s	remaining: 4m 17s
5:	learn: 4.0796907	total: 2.6s	remaining: 4m 17s
6:	learn: 3.9946240	total: 3.04s	remaining: 4m 17s
7:	learn: 3.9165634	total: 3.47s	remaining: 4m 16s
8:	learn: 3.8382479	total: 3.9s	remaining: 4m 15s
9:	learn: 3.7639682	total: 4.33s	remaining: 4m 15s
10:	learn: 3.6902956	total: 4.76s	remaining: 4m 14s
11:	learn: 3.6232998	total: 5.19s	remaining: 4m 14s
12:	learn: 3.5561972	total: 5.63s	remaining: 4m 14s
13:	learn: 3.4930392	total: 6.06s	remaining: 4m 13s
14:	learn: 3.4318394	total: 6.49s	remaining: 4m 13s
15:	learn: 3.3723144	total: 6.94s	remaining: 4m 13s
16:	learn: 3.3145891	total: 7.37s	remaining: 4m 12s
17:	learn: 3.2574311	total: 7.8s	remaining: 4m 12s
18:	learn: 3.2034243	total: 8.23s	remaining: 4m 11s
19:	learn: 3.1502195	total

158:	learn: 1.7441794	total: 1m 9s	remaining: 3m 11s
159:	learn: 1.7427540	total: 1m 9s	remaining: 3m 11s
160:	learn: 1.7416752	total: 1m 9s	remaining: 3m 10s
161:	learn: 1.7407640	total: 1m 10s	remaining: 3m 10s
162:	learn: 1.7396505	total: 1m 10s	remaining: 3m 9s
163:	learn: 1.7384088	total: 1m 11s	remaining: 3m 9s
164:	learn: 1.7374372	total: 1m 11s	remaining: 3m 8s
165:	learn: 1.7364671	total: 1m 12s	remaining: 3m 8s
166:	learn: 1.7354811	total: 1m 12s	remaining: 3m 8s
167:	learn: 1.7345899	total: 1m 12s	remaining: 3m 7s
168:	learn: 1.7334050	total: 1m 13s	remaining: 3m 7s
169:	learn: 1.7324095	total: 1m 13s	remaining: 3m 6s
170:	learn: 1.7314042	total: 1m 14s	remaining: 3m 6s
171:	learn: 1.7301985	total: 1m 14s	remaining: 3m 5s
172:	learn: 1.7292433	total: 1m 15s	remaining: 3m 5s
173:	learn: 1.7278725	total: 1m 15s	remaining: 3m 5s
174:	learn: 1.7267342	total: 1m 15s	remaining: 3m 4s
175:	learn: 1.7260769	total: 1m 16s	remaining: 3m 4s
176:	learn: 1.7251228	total: 1m 16s	remaining

312:	learn: 1.6365771	total: 2m 16s	remaining: 2m 4s
313:	learn: 1.6361398	total: 2m 16s	remaining: 2m 4s
314:	learn: 1.6358587	total: 2m 16s	remaining: 2m 3s
315:	learn: 1.6352320	total: 2m 17s	remaining: 2m 3s
316:	learn: 1.6349130	total: 2m 17s	remaining: 2m 3s
317:	learn: 1.6346229	total: 2m 18s	remaining: 2m 2s
318:	learn: 1.6338700	total: 2m 18s	remaining: 2m 2s
319:	learn: 1.6335754	total: 2m 19s	remaining: 2m 1s
320:	learn: 1.6333602	total: 2m 19s	remaining: 2m 1s
321:	learn: 1.6330817	total: 2m 19s	remaining: 2m
322:	learn: 1.6327756	total: 2m 20s	remaining: 2m
323:	learn: 1.6324407	total: 2m 20s	remaining: 1m 59s
324:	learn: 1.6319022	total: 2m 21s	remaining: 1m 59s
325:	learn: 1.6315501	total: 2m 21s	remaining: 1m 59s
326:	learn: 1.6311455	total: 2m 22s	remaining: 1m 58s
327:	learn: 1.6308463	total: 2m 22s	remaining: 1m 58s
328:	learn: 1.6302281	total: 2m 22s	remaining: 1m 57s
329:	learn: 1.6298678	total: 2m 23s	remaining: 1m 57s
330:	learn: 1.6296062	total: 2m 23s	remaining

466:	learn: 1.5867882	total: 3m 22s	remaining: 57.8s
467:	learn: 1.5866371	total: 3m 23s	remaining: 57.4s
468:	learn: 1.5865289	total: 3m 23s	remaining: 56.9s
469:	learn: 1.5863363	total: 3m 24s	remaining: 56.5s
470:	learn: 1.5860279	total: 3m 24s	remaining: 56s
471:	learn: 1.5858771	total: 3m 25s	remaining: 55.6s
472:	learn: 1.5856707	total: 3m 25s	remaining: 55.2s
473:	learn: 1.5852824	total: 3m 25s	remaining: 54.7s
474:	learn: 1.5850516	total: 3m 26s	remaining: 54.3s
475:	learn: 1.5849501	total: 3m 26s	remaining: 53.9s
476:	learn: 1.5846100	total: 3m 27s	remaining: 53.4s
477:	learn: 1.5843326	total: 3m 27s	remaining: 53s
478:	learn: 1.5840792	total: 3m 28s	remaining: 52.6s
479:	learn: 1.5838777	total: 3m 28s	remaining: 52.1s
480:	learn: 1.5834653	total: 3m 28s	remaining: 51.7s
481:	learn: 1.5832127	total: 3m 29s	remaining: 51.3s
482:	learn: 1.5828986	total: 3m 29s	remaining: 50.8s
483:	learn: 1.5824871	total: 3m 30s	remaining: 50.4s
484:	learn: 1.5820749	total: 3m 30s	remaining: 50s

In [248]:
pre_li = lig_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
pre_xg = xgb_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
pre_ca = cat_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
print(
    (((abs(city_tainan_test_ans - pre_li ) / city_tainan_test_ans ) < 0.1 ).sum() ) / len(city_tainan_test_ans),
    (((abs(city_tainan_test_ans - pre_xg ) / city_tainan_test_ans ) < 0.1 ).sum() ) / len(city_tainan_test_ans),
    (((abs(city_tainan_test_ans - pre_ca ) / city_tainan_test_ans ) < 0.1 ).sum() ) / len(city_tainan_test_ans)
     )

0.5214800878018188 0.5360614612731264 0.49827532141737224


In [220]:
pre = lig_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
print('hit_rate',
    (((abs(city_tainan_test_ans - pre ) / city_tainan_test_ans ) < 0.1 ).sum() ) / len(city_tainan_test_ans))

hit_rate 0.5214800878018188


In [249]:
city_taipei_test_ans = city_taipei_test['BUILD_U_PRICE.ADJ']
city_taipei_test = city_taipei_test.drop('BUILD_U_PRICE.ADJ',axis=1)
city_taipei_all = pd.concat([city_taipei_train,city_taipei_test])
train,test = transform(city_taipei_all)

xgb_model = xgb.XGBRegressor(n_estimators=600,objective = 'reg:squarederror',subsample=0.8,
                             max_depth=10,learning_rate=0.01,gamma=0.95,
                            colsample_bytree=0.99,reg_lambda=0.7,min_child_weight=0.9,
                             reg_alpha=0,n_jobs=5,verbosity=2,seed=1102)


light_model = lgb.LGBMRegressor(boosting_type='gbdt',learning_rate=0.01,num_leaves=100,max_depth=15,subsample=0.88,
                                    colsample_bytree=0.4,reg_alpha=0.1,reg_lambda=0.4,n_estimators=600,n_jobs=5)


cat_model = cat.CatBoostRegressor(n_estimators=600,one_hot_max_size=2, depth=8, l2_leaf_reg=3,
                                 random_seed=1102)


forest_model = RandomForestRegressor(n_estimators=200,n_jobs=5,max_depth=12,random_state=1102)


##### for xgboost 

X_train, X_test, y_train, y_test = train_test_split(train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1), train['BUILD_U_PRICE.ADJ'], test_size=0.2, random_state=1102)    

################################## for xgboost 


print(X_train.shape)



xgb_mo = xgb_model.fit(X_train,y_train)
lig_mo = light_model.fit(X_train,y_train)
cat_mo = cat_model.fit(X_train,y_train)
#rf_mo = forest_model.fit(X_train,y_train)

#f_cv = KFold(n_splits=5)

#cv = cross_val_score(light_model,X_train,y_train,cv=f_cv,n_jobs=5)

(61496, 154)
(61496, 154)
(42190, 195)
[16:51:24] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 206 extra nodes, 0 pruned nodes, max_depth=10
[16:51:25] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 218 extra nodes, 2 pruned nodes, max_depth=10
[16:51:25] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 232 extra nodes, 0 pruned nodes, max_depth=10
[16:51:25] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 246 extra nodes, 0 pruned nodes, max_depth=10
[16:51:25] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 250 extra nodes, 0 pruned nodes, max_depth=10
[16:51:26] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots,

[16:51:37] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 528 extra nodes, 2 pruned nodes, max_depth=10
[16:51:37] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 498 extra nodes, 4 pruned nodes, max_depth=10
[16:51:38] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 486 extra nodes, 0 pruned nodes, max_depth=10
[16:51:38] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 486 extra nodes, 0 pruned nodes, max_depth=10
[16:51:38] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 502 extra nodes, 0 pruned nodes, max_depth=10
[16:51:39] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 528 extra nodes, 0 pruned nodes, max_d

[16:51:51] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 824 extra nodes, 2 pruned nodes, max_depth=10
[16:51:51] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 818 extra nodes, 2 pruned nodes, max_depth=10
[16:51:51] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 816 extra nodes, 10 pruned nodes, max_depth=10
[16:51:52] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 836 extra nodes, 4 pruned nodes, max_depth=10
[16:51:52] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 782 extra nodes, 4 pruned nodes, max_depth=10
[16:51:52] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 810 extra nodes, 2 pruned nodes, max_

[16:52:05] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1048 extra nodes, 14 pruned nodes, max_depth=10
[16:52:05] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 948 extra nodes, 10 pruned nodes, max_depth=10
[16:52:05] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 994 extra nodes, 18 pruned nodes, max_depth=10
[16:52:05] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1038 extra nodes, 10 pruned nodes, max_depth=10
[16:52:06] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 958 extra nodes, 14 pruned nodes, max_depth=10
[16:52:06] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1064 extra nodes, 12 pruned nod

[16:52:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1078 extra nodes, 14 pruned nodes, max_depth=10
[16:52:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1100 extra nodes, 24 pruned nodes, max_depth=10
[16:52:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1094 extra nodes, 18 pruned nodes, max_depth=10
[16:52:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1066 extra nodes, 20 pruned nodes, max_depth=10
[16:52:20] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1114 extra nodes, 20 pruned nodes, max_depth=10
[16:52:20] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 852 extra nodes, 30 pruned n

...

[16:54:10] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 952 extra nodes, 48 pruned nodes, max_depth=10
[16:54:11] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1120 extra nodes, 34 pruned nodes, max_depth=10
[16:54:11] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 614 extra nodes, 44 pruned nodes, max_depth=10
[16:54:11] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1070 extra nodes, 12 pruned nodes, max_depth=10
[16:54:12] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 884 extra nodes, 30 pruned nodes, max_depth=10
[16:54:12] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 754 extra nodes, 34 pruned node

119:	learn: 7.8023919	total: 1m 11s	remaining: 4m 44s
120:	learn: 7.7930616	total: 1m 11s	remaining: 4m 44s
121:	learn: 7.7837924	total: 1m 12s	remaining: 4m 43s
122:	learn: 7.7734554	total: 1m 12s	remaining: 4m 42s
123:	learn: 7.7660653	total: 1m 13s	remaining: 4m 42s
124:	learn: 7.7532754	total: 1m 14s	remaining: 4m 41s
125:	learn: 7.7423739	total: 1m 14s	remaining: 4m 41s
126:	learn: 7.7313131	total: 1m 15s	remaining: 4m 40s
127:	learn: 7.7220557	total: 1m 16s	remaining: 4m 40s
128:	learn: 7.7133834	total: 1m 16s	remaining: 4m 39s
129:	learn: 7.7054431	total: 1m 17s	remaining: 4m 39s
130:	learn: 7.6980934	total: 1m 17s	remaining: 4m 38s
131:	learn: 7.6907985	total: 1m 18s	remaining: 4m 37s
132:	learn: 7.6847914	total: 1m 18s	remaining: 4m 37s
133:	learn: 7.6760229	total: 1m 19s	remaining: 4m 37s
134:	learn: 7.6674041	total: 1m 20s	remaining: 4m 36s
135:	learn: 7.6555045	total: 1m 20s	remaining: 4m 36s
136:	learn: 7.6489667	total: 1m 21s	remaining: 4m 35s
137:	learn: 7.6426638	total:

272:	learn: 7.1318484	total: 2m 41s	remaining: 3m 13s
273:	learn: 7.1302248	total: 2m 42s	remaining: 3m 13s
274:	learn: 7.1285963	total: 2m 42s	remaining: 3m 12s
275:	learn: 7.1265284	total: 2m 43s	remaining: 3m 11s
276:	learn: 7.1243205	total: 2m 44s	remaining: 3m 11s
277:	learn: 7.1220273	total: 2m 44s	remaining: 3m 10s
278:	learn: 7.1192228	total: 2m 45s	remaining: 3m 10s
279:	learn: 7.1164655	total: 2m 45s	remaining: 3m 9s
280:	learn: 7.1147986	total: 2m 46s	remaining: 3m 9s
281:	learn: 7.1129056	total: 2m 47s	remaining: 3m 8s
282:	learn: 7.1105119	total: 2m 47s	remaining: 3m 7s
283:	learn: 7.1085290	total: 2m 48s	remaining: 3m 7s
284:	learn: 7.1063426	total: 2m 48s	remaining: 3m 6s
285:	learn: 7.1045074	total: 2m 49s	remaining: 3m 6s
286:	learn: 7.1015196	total: 2m 50s	remaining: 3m 5s
287:	learn: 7.0993866	total: 2m 50s	remaining: 3m 4s
288:	learn: 7.0960315	total: 2m 51s	remaining: 3m 4s
289:	learn: 7.0941706	total: 2m 51s	remaining: 3m 3s
290:	learn: 7.0922678	total: 2m 52s	rem

426:	learn: 6.9045641	total: 4m 14s	remaining: 1m 43s
427:	learn: 6.9036153	total: 4m 15s	remaining: 1m 42s
428:	learn: 6.9029282	total: 4m 15s	remaining: 1m 41s
429:	learn: 6.9018707	total: 4m 16s	remaining: 1m 41s
430:	learn: 6.9009635	total: 4m 16s	remaining: 1m 40s
431:	learn: 6.8988365	total: 4m 17s	remaining: 1m 40s
432:	learn: 6.8968780	total: 4m 18s	remaining: 1m 39s
433:	learn: 6.8959303	total: 4m 18s	remaining: 1m 38s
434:	learn: 6.8946080	total: 4m 19s	remaining: 1m 38s
435:	learn: 6.8939336	total: 4m 19s	remaining: 1m 37s
436:	learn: 6.8931632	total: 4m 20s	remaining: 1m 37s
437:	learn: 6.8919768	total: 4m 21s	remaining: 1m 36s
438:	learn: 6.8907834	total: 4m 21s	remaining: 1m 35s
439:	learn: 6.8896092	total: 4m 22s	remaining: 1m 35s
440:	learn: 6.8888794	total: 4m 22s	remaining: 1m 34s
441:	learn: 6.8879570	total: 4m 23s	remaining: 1m 34s
442:	learn: 6.8868979	total: 4m 24s	remaining: 1m 33s
443:	learn: 6.8845380	total: 4m 24s	remaining: 1m 32s
444:	learn: 6.8832577	total:

581:	learn: 6.7479583	total: 5m 45s	remaining: 10.7s
582:	learn: 6.7471290	total: 5m 45s	remaining: 10.1s
583:	learn: 6.7466430	total: 5m 46s	remaining: 9.49s
584:	learn: 6.7460660	total: 5m 46s	remaining: 8.9s
585:	learn: 6.7453152	total: 5m 47s	remaining: 8.3s
586:	learn: 6.7442981	total: 5m 48s	remaining: 7.71s
587:	learn: 6.7434991	total: 5m 48s	remaining: 7.12s
588:	learn: 6.7428690	total: 5m 49s	remaining: 6.52s
589:	learn: 6.7414167	total: 5m 49s	remaining: 5.93s
590:	learn: 6.7406183	total: 5m 50s	remaining: 5.34s
591:	learn: 6.7401752	total: 5m 51s	remaining: 4.75s
592:	learn: 6.7393692	total: 5m 51s	remaining: 4.15s
593:	learn: 6.7385872	total: 5m 52s	remaining: 3.56s
594:	learn: 6.7379209	total: 5m 53s	remaining: 2.97s
595:	learn: 6.7359832	total: 5m 53s	remaining: 2.37s
596:	learn: 6.7350725	total: 5m 54s	remaining: 1.78s
597:	learn: 6.7341478	total: 5m 55s	remaining: 1.19s
598:	learn: 6.7318105	total: 5m 55s	remaining: 594ms
599:	learn: 6.7309247	total: 5m 56s	remaining: 0

In [250]:
pre_li = lig_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
pre_xg = xgb_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
pre_ca = cat_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
print(
    (((abs(city_taipei_test_ans - pre_li ) / city_taipei_test_ans ) < 0.1 ).sum() ) / len(city_taipei_test_ans),
    (((abs(city_taipei_test_ans - pre_xg ) / city_taipei_test_ans ) < 0.1 ).sum() ) / len(city_taipei_test_ans),
    (((abs(city_taipei_test_ans - pre_ca ) / city_taipei_test_ans ) < 0.1 ).sum() ) / len(city_taipei_test_ans)
     )

0.6454670015985384 0.6508335236355333 0.6174925782142041


In [222]:
pre = lig_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
print('hit_rate',
    (((abs(city_taipei_test_ans - pre ) / city_taipei_test_ans ) < 0.1 ).sum() ) / len(city_taipei_test_ans))

hit_rate 0.6454670015985384


In [251]:
city_taoyuan_test_ans = city_taoyuan_test['BUILD_U_PRICE.ADJ']
city_taoyuan_test = city_taoyuan_test.drop('BUILD_U_PRICE.ADJ',axis=1)
city_taoyuan_all = pd.concat([city_taoyuan_train,city_taoyuan_test])
train,test = transform(city_taoyuan_all)

xgb_model = xgb.XGBRegressor(n_estimators=600,objective = 'reg:squarederror',subsample=0.8,
                             max_depth=10,learning_rate=0.01,gamma=0.95,
                            colsample_bytree=0.99,reg_lambda=0.7,min_child_weight=0.9,
                             reg_alpha=0,n_jobs=5,verbosity=2,seed=1102)


light_model = lgb.LGBMRegressor(boosting_type='gbdt',learning_rate=0.01,num_leaves=100,max_depth=15,subsample=0.88,
                                    colsample_bytree=0.4,reg_alpha=0.1,reg_lambda=0.4,n_estimators=600,n_jobs=5)


cat_model = cat.CatBoostRegressor(n_estimators=600,one_hot_max_size=2, depth=8, l2_leaf_reg=3,
                                 random_seed=1102)


forest_model = RandomForestRegressor(n_estimators=200,n_jobs=5,max_depth=12,random_state=1102)


##### for xgboost 

X_train, X_test, y_train, y_test = train_test_split(train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1), train['BUILD_U_PRICE.ADJ'], test_size=0.2, random_state=1102)    

################################## for xgboost 


print(X_train.shape)



xgb_mo = xgb_model.fit(X_train,y_train)
lig_mo = light_model.fit(X_train,y_train)
cat_mo = cat_model.fit(X_train,y_train)
#rf_mo = forest_model.fit(X_train,y_train)

#f_cv = KFold(n_splits=5)

#cv = cross_val_score(light_model,X_train,y_train,cv=f_cv,n_jobs=5)

(103001, 154)
(103001, 154)
(69224, 198)
[17:00:54] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 488 extra nodes, 8 pruned nodes, max_depth=10
[17:00:54] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 484 extra nodes, 14 pruned nodes, max_depth=10
[17:00:55] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 484 extra nodes, 12 pruned nodes, max_depth=10
[17:00:55] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 512 extra nodes, 18 pruned nodes, max_depth=10
[17:00:56] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 492 extra nodes, 16 pruned nodes, max_depth=10
[17:00:56] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 

[17:01:16] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 746 extra nodes, 34 pruned nodes, max_depth=10
[17:01:17] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 758 extra nodes, 32 pruned nodes, max_depth=10
[17:01:17] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 708 extra nodes, 26 pruned nodes, max_depth=10
[17:01:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 762 extra nodes, 24 pruned nodes, max_depth=10
[17:01:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 762 extra nodes, 30 pruned nodes, max_depth=10
[17:01:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 796 extra nodes, 24 pruned nodes,

[17:01:39] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1030 extra nodes, 32 pruned nodes, max_depth=10
[17:01:40] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1006 extra nodes, 32 pruned nodes, max_depth=10
[17:01:40] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1044 extra nodes, 48 pruned nodes, max_depth=10
[17:01:41] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1132 extra nodes, 52 pruned nodes, max_depth=10
[17:01:41] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1064 extra nodes, 44 pruned nodes, max_depth=10
[17:01:42] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1046 extra nodes, 54 pruned 

[17:02:03] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1236 extra nodes, 46 pruned nodes, max_depth=10
[17:02:03] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1150 extra nodes, 58 pruned nodes, max_depth=10
[17:02:04] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1136 extra nodes, 58 pruned nodes, max_depth=10
[17:02:04] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1220 extra nodes, 74 pruned nodes, max_depth=10
[17:02:05] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1170 extra nodes, 62 pruned nodes, max_depth=10
[17:02:05] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1120 extra nodes, 62 pruned 

[17:02:26] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1202 extra nodes, 78 pruned nodes, max_depth=10
[17:02:27] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1170 extra nodes, 52 pruned nodes, max_depth=10
[17:02:27] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1120 extra nodes, 92 pruned nodes, max_depth=10
[17:02:28] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1238 extra nodes, 86 pruned nodes, max_depth=10
[17:02:28] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1288 extra nodes, 68 pruned nodes, max_depth=10
[17:02:29] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1274 extra nodes, 84 pruned 

...

[17:05:31] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 796 extra nodes, 126 pruned nodes, max_depth=10
[17:05:31] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1082 extra nodes, 186 pruned nodes, max_depth=10
[17:05:32] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 850 extra nodes, 106 pruned nodes, max_depth=10
[17:05:32] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 680 extra nodes, 150 pruned nodes, max_depth=10
[17:05:33] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 872 extra nodes, 106 pruned nodes, max_depth=10
[17:05:33] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1046 extra nodes, 146 prune

120:	learn: 2.2026129	total: 15.6s	remaining: 1m 1s
121:	learn: 2.2006656	total: 15.7s	remaining: 1m 1s
122:	learn: 2.1987403	total: 15.8s	remaining: 1m 1s
123:	learn: 2.1963746	total: 16s	remaining: 1m 1s
124:	learn: 2.1943625	total: 16.1s	remaining: 1m 1s
125:	learn: 2.1925965	total: 16.2s	remaining: 1m
126:	learn: 2.1905543	total: 16.3s	remaining: 1m
127:	learn: 2.1886358	total: 16.5s	remaining: 1m
128:	learn: 2.1867258	total: 16.6s	remaining: 1m
129:	learn: 2.1851401	total: 16.7s	remaining: 1m
130:	learn: 2.1836943	total: 16.8s	remaining: 1m
131:	learn: 2.1819769	total: 17s	remaining: 1m
132:	learn: 2.1796588	total: 17.1s	remaining: 1m
133:	learn: 2.1780548	total: 17.2s	remaining: 59.9s
134:	learn: 2.1767875	total: 17.3s	remaining: 59.7s
135:	learn: 2.1753209	total: 17.5s	remaining: 59.6s
136:	learn: 2.1734850	total: 17.6s	remaining: 59.5s
137:	learn: 2.1719521	total: 17.7s	remaining: 59.3s
138:	learn: 2.1705175	total: 17.9s	remaining: 59.2s
139:	learn: 2.1685839	total: 18s	remaini

280:	learn: 2.0543554	total: 36s	remaining: 40.8s
281:	learn: 2.0540329	total: 36.1s	remaining: 40.7s
282:	learn: 2.0535332	total: 36.2s	remaining: 40.6s
283:	learn: 2.0528780	total: 36.4s	remaining: 40.4s
284:	learn: 2.0525330	total: 36.5s	remaining: 40.3s
285:	learn: 2.0519808	total: 36.6s	remaining: 40.2s
286:	learn: 2.0515245	total: 36.7s	remaining: 40.1s
287:	learn: 2.0507036	total: 36.9s	remaining: 39.9s
288:	learn: 2.0504022	total: 37s	remaining: 39.8s
289:	learn: 2.0497686	total: 37.1s	remaining: 39.7s
290:	learn: 2.0492846	total: 37.2s	remaining: 39.5s
291:	learn: 2.0488831	total: 37.4s	remaining: 39.4s
292:	learn: 2.0478630	total: 37.5s	remaining: 39.3s
293:	learn: 2.0473537	total: 37.6s	remaining: 39.2s
294:	learn: 2.0464771	total: 37.7s	remaining: 39s
295:	learn: 2.0455672	total: 37.9s	remaining: 38.9s
296:	learn: 2.0452250	total: 38s	remaining: 38.8s
297:	learn: 2.0445381	total: 38.1s	remaining: 38.6s
298:	learn: 2.0440566	total: 38.3s	remaining: 38.5s
299:	learn: 2.043340

440:	learn: 1.9731803	total: 56.2s	remaining: 20.3s
441:	learn: 1.9727790	total: 56.3s	remaining: 20.1s
442:	learn: 1.9724136	total: 56.5s	remaining: 20s
443:	learn: 1.9721112	total: 56.6s	remaining: 19.9s
444:	learn: 1.9712364	total: 56.7s	remaining: 19.8s
445:	learn: 1.9708379	total: 56.9s	remaining: 19.6s
446:	learn: 1.9705109	total: 57s	remaining: 19.5s
447:	learn: 1.9700210	total: 57.1s	remaining: 19.4s
448:	learn: 1.9697390	total: 57.2s	remaining: 19.2s
449:	learn: 1.9693701	total: 57.3s	remaining: 19.1s
450:	learn: 1.9690717	total: 57.5s	remaining: 19s
451:	learn: 1.9687273	total: 57.6s	remaining: 18.9s
452:	learn: 1.9683154	total: 57.7s	remaining: 18.7s
453:	learn: 1.9679375	total: 57.8s	remaining: 18.6s
454:	learn: 1.9674946	total: 58s	remaining: 18.5s
455:	learn: 1.9673531	total: 58.1s	remaining: 18.3s
456:	learn: 1.9668409	total: 58.2s	remaining: 18.2s
457:	learn: 1.9662126	total: 58.3s	remaining: 18.1s
458:	learn: 1.9660463	total: 58.5s	remaining: 18s
459:	learn: 1.9657798	

598:	learn: 1.9151874	total: 1m 16s	remaining: 127ms
599:	learn: 1.9149955	total: 1m 16s	remaining: 0us


In [252]:
pre_li = lig_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
pre_xg = xgb_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
pre_ca = cat_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
print(
    (((abs(city_taoyuan_test_ans - pre_li ) / city_taoyuan_test_ans ) < 0.1 ).sum() ) / len(city_taoyuan_test_ans),
    (((abs(city_taoyuan_test_ans - pre_xg ) / city_taoyuan_test_ans ) < 0.1 ).sum() ) / len(city_taoyuan_test_ans),
    (((abs(city_taoyuan_test_ans - pre_ca ) / city_taoyuan_test_ans ) < 0.1 ).sum() ) / len(city_taoyuan_test_ans)
     )

0.6483316112563059 0.6426183674709779 0.6250531817905549


In [224]:
pre = lig_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
print('hit_rate',
    (((abs(city_taoyuan_test_ans - pre ) / city_taoyuan_test_ans ) < 0.1 ).sum() ) / len(city_taoyuan_test_ans))

hit_rate 0.6483316112563059


In [225]:
train_set = data_all_re[data_all_re['TRADE_YM'] <= 10700]
test_set = data_all_re[data_all_re['TRADE_YM'] > 10700]

In [226]:
test_set_ans = test_set['BUILD_U_PRICE.ADJ']
test_set = test_set.drop('BUILD_U_PRICE.ADJ',axis=1)
train_all = pd.concat([train_set,test_set])
train,test = transform(train_all)

xgb_model = xgb.XGBRegressor(n_estimators=600,objective = 'reg:squarederror',subsample=0.8,
                             max_depth=10,learning_rate=0.01,gamma=0.95,
                            colsample_bytree=0.99,reg_lambda=0.7,min_child_weight=0.9,
                             reg_alpha=0,n_jobs=5,verbosity=2,seed=1102)


light_model = lgb.LGBMRegressor(boosting_type='gbdt',learning_rate=0.01,num_leaves=100,max_depth=15,subsample=0.88,
                                    colsample_bytree=0.4,reg_alpha=0.1,reg_lambda=0.4,n_estimators=600,n_jobs=5)


cat_model = cat.CatBoostRegressor(n_estimators=600,one_hot_max_size=2, depth=8, l2_leaf_reg=3,
                                 random_seed=1102)


forest_model = RandomForestRegressor(n_estimators=200,n_jobs=5,max_depth=12,random_state=1102)


##### for xgboost 

X_train, X_test, y_train, y_test = train_test_split(train.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1), train['BUILD_U_PRICE.ADJ'], test_size=0.2, random_state=1102)    

################################## for xgboost 


print(X_train.shape)



xgb_mo = xgb_model.fit(X_train,y_train)
lig_mo = light_model.fit(X_train,y_train)
cat_mo = cat_model.fit(X_train,y_train)
#rf_mo = forest_model.fit(X_train,y_train)

#f_cv = KFold(n_splits=5)

#cv = cross_val_score(light_model,X_train,y_train,cv=f_cv,n_jobs=5)

(573415, 155)
(573415, 155)
(381159, 201)


In [227]:
pre = lig_mo.predict(test.drop(['BUILD_U_PRICE.ADJ','BUILD_U_PRICE'],axis=1))
print('hit_rate',
    (((abs(test_set_ans - pre ) / test_set_ans ) < 0.1 ).sum() ) / len(test_set_ans))

hit_rate 0.5685843448105028


In [253]:
print(
city_hsin_test.shape,
city_kao_test.shape,
city_newtaipei_test.shape,
city_taichung_test.shape,
city_tainan_test.shape,
city_taipei_test.shape,
city_taoyuan_test.shape)

(7287, 169) (14871, 169) (27755, 169) (15386, 169) (6378, 169) (8758, 169) (16453, 169)


In [ ]:
def my_logistic_obj(y_hat, dtrain):
    y = dtrain.get_label()
    p = y_hat 
    grad = 4 * p * y + p - 5 * y
    hess = (4 * y + 1) * (p * (1.0 - p))
    return grad, hess

def my_err_rate(y_hat, dtrain):
    y = dtrain.get_label()
    y_hat = np.clip(y_hat, 10e-7, 1-10e-7)
    loss_fn = y*np.log(y_hat)
    loss_fp = (1.0 - y)*np.log(1.0 - y_hat)
    return 'myloss', np.sum(-(5*loss_fn+loss_fp))/len(y), False

def calc_loss(y, yp): #same as my_err_rate
    yp = np.clip(yp, 10e-7, 1.0-10e-7)
    loss_fn = y*np.log(yp)
    loss_fp = (1.0-y)*np.log(1.0-yp)
    return np.sum(-(5*loss_fn+loss_fp))/y.shape[0]

params = {
    'task': 'train',
    'objective': 'regression',
    'boosting': 'gbdt',
    'metric': 'auc',
    'train_metric': '+',
    'num_leaves': 260,
    'learning_rate': 0.0245,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'max_depth': 15,
    'max_bin': 512
}

categoricals = None
lgb_train = lgb.Dataset(X_trn, y_trn, feature_name=features, categorical_feature=categoricals, free_raw_data=True) 
lgb_eval = lgb.Dataset(X_val, y_val, feature_name=features, categorical_feature=categoricals, reference=lgb_train, free_raw_data=True)

evals_result = {}
gbm = lgb.train(params,
                lgb_train,
                num_boost_round = 5,
                valid_sets=[lgb_eval, lgb_train],
#                 early_stopping_rounds = 2,
#                 fobj = my_logistic_obj, # <~~ First without fobj
                feval = my_err_rate,
                evals_result=evals_result,
                verbose_eval=1
               )

In [ ]:
def evaluate_macroF1_lgb(truth, predictions):  
    pred_labels = predictions.reshape(len(np.unique(truth)),-1).argmax(axis=0)
    f1 = f1_score(truth, pred_labels, average='macro')
    return ('macroF1', f1, True) 

lg = LGBMClassifier(n_estimators=1000)

lg.fit(x_train,y_train,eval_set=(x_test,y_test),eval_metric=evaluate_macroF1_lgb,early_stopping_rounds=25)